# Imports

In [1]:
%reload_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.transforms as transforms
import juliet
import seaborn as sns
import mplcyberpunk
import numpy as np
import scipy as sp
import pandas as pd
import glob
import json
import os
import pickle
import corner
import pathlib
import itertools
import batman
import json
import re

from astropy.io import fits, ascii
from astropy import constants as c
from astropy import units as u
from astropy.time import Time
from matplotlib.collections import LineCollection
from tqdm import tqdm
from datetime import datetime, timedelta
from dateutil import parser
from PyAstronomy.pyTiming import pyPeriod

import utils

In [2]:
#####################
# Display and backend
#####################
FIG_LARGE = (11, 8) # Default plot figure size for large figures
FIG_WIDE = (11, 5)    
%config InlineBackend.figure_format = "retina" # Crisp retina display on macs
# Qt5 backend for interactive plotting
%matplotlib qt5 

################
# Theme settings
################
def set_theme(notebook_mode="dark"):
    plt.style.use("default") # Reset to default before layering on any changes
    sns.set(palette="Paired", color_codes=True, context="talk")

    if notebook_mode.lower() == "paper":
        sns.set_style('ticks')
        params = {
            # xticks
            "xtick.top":False,
            "xtick.direction":"out",
            "xtick.major.size":5,
            "xtick.minor.visible":False,

            # yticks
            "ytick.right":False,
            "ytick.direction":"out",
            "ytick.major.size":5,
            "ytick.minor.visible":False,

            # pallete
            "axes.prop_cycle":mpl.cycler(color=[
                "#fdbf6f", # Yellow
                "#ff7f00", # Orange
                "#a6cee3", # Cyan
                "#1f78b4", # Blue
                "#956cb4", # Purple
                "#029e73", # Green
                "#c44e52", # Red
            ]),
        }
        #sns.set_style("ticks", tick_params)
        #params = {
        #    "axes.formatter.limits":(-3, 7),
        #    #"axes.spine.right":False,
        #    "xtick.major.size":2,
        #}
        plt.rcParams.update(params)

    elif notebook_mode.lower() == "dark":
        sns.set(palette="colorblind", color_codes=True, context="talk")
        plt.style.use("cyberpunk")
        # Re-order color cycle
        params = {
            "axes.prop_cycle":mpl.cycler(color=[
                "#F5D300", # Yellow
                'r',       # Red
                "#08F7FE", # Cyan
                "b", # Blue
                "g", # Green
            ]),
        }
        plt.rcParams.update(params)

    else:
        plt.style.use("default")
    
HOURS = mdates.HourLocator() # For UTC plots with ticks every hour
pd.set_option("display.max_rows", None, "display.max_columns", None)

from tqdm import tqdm_notebook as tqdm

set_theme("dark")

In [ ]:
sns.palplot(sns.palettes.color_palette('muted'))

In [ ]:
sns.palettes.color_palette('muted').as_hex()

## Scale Height

In [ ]:
print()

In [ ]:
def get_Teq(Ts=None, albedo=None, aRs=None, Rs=None, a=None):
    if (aRs is None):
        return Ts * (1 - albedo)**0.25 * (0.5*Rs/a)**0.5
    else:
        return Ts * (1 - albedo)**0.25 * (0.5/aRs)**0.5

def get_H(
    Tp=None,
    Mp=None,
    Rp=None,
    Ts=None,
    Rs=None,
    mu=None,
    albedo=None,
    aRs=None,
    a=None,
    RpRs=None,
):
    if isinstance(Mp, u.Quantity):
        G_Mp = c.G * Mp
    else:
        # Default to Jupiter mass
        G_Mp = c.GM_jup * Mp
    if (RpRs is None) and (Rp is not None):
        g = G_Mp / Rp**2
    else:
        g = G_Mp / (RpRs**2 * Rs**2)
    if Tp is None:
        Tp = get_Teq(Ts=Ts, albedo=0, aRs=aRs, Rs=Rs, a=a)
    return c.k_B * Tp / (mu * g), Tp

def get_depth(Tp=None,
              Mp=None,
              Rp=None,
              Ts=None,
              Rs=None,
              mu=None,
              H=None,
              aRs=None,
              a=None,
              RpRs=None,
):
    if H is None:
        H, Tp = get_H(Tp=Tp, Mp=Mp, Rp=Rp, Ts=Ts, Rs=Rs, mu=mu, aRs=aRs, a=a, RpRs=RpRs)
    if RpRs is None:
        Delta_D =  2 * H * Rp / Rs**2, H, Tp
    else:
        Delta_D =  2 * H * RpRs/Rs, H, Tp
    return Delta_D

D, H, Tp = get_depth(
    #Rp=1.036*u.R_jup,
    mu=2*u.Da,
    Mp=1.97*u.M_jup,
    Ts=5734*u.K,
    Rs=1.1858169*u.R_sun,
    RpRs = 0.1113,
    aRs=4.26,
    #a=0.01528*u.AU,
    #Tp=1440*u.K,
)

print(f'Tp = {Tp.cgs:.3f}')
print(f'H = {H.to("km"):.3f},')
n = 5
print(f'Delta D = {n * D.cgs * 1e6:.3f} ppm at {n} scale heights')

# Raw data inspection

In [ ]:
fpaths = glob.glob("./data/HATP23/ut160720/ift*c1.fits")
#fpaths = glob("./data/HATP23/ut160621/ift*c1.fits")

object_names = []
for fpath in fpaths:
    with open(fpath, "rb") as f:
        header = fits.open(f, de)[0].header
        name = header["OBJECT"]
        object_names.append(name)
        
len(object_names)

In [ ]:
# Config:
images_dir = '../atmospheres_meeting/figs'
result_grid_filename = '/Users/mango/Desktop/grid.png'
result_figsize_resolution = 40 # 1 = 100px

images_list = glob.glob(f"{images_dir}/*raw_lc.png")

# Create plt plot:
fig, axes = plt.subplots(5, 2, figsize=(8, 40))
axes = axes.flatten()

for ax, img in zip(axes, images_list):
    plt_image = plt.imread(img)
    plt_image = np.array(Image.open(img))
    ax.imshow(plt_image, aspect="auto")
    ax.axis("off")
    
fig.tight_layout()
plt.subplots_adjust(wspace=.0, hspace=.0)
#plt.savefig("/Users/mango/Desktop/grid.png", dpi=250, bbox_inches="tight", pad_inches=0)

## File stats

In [3]:
data_dir = './data/HATP26/ut190313'
data = data_dir.split('/')[-1]
fpaths = np.array(sorted(glob.glob(f'{data_dir}/ift*c1.fits')))
filenames = []
objects = []
slits = []
airmasses = []
file_info = {}
for fpath in tqdm(fpaths):
    header = utils.fits_header(fpath)
    #header = fits.getheader(fpath)
    #print(header['OBJECT'])
    if header is not None:
        file_info[header['FILENAME'].split('c')[0]] = {
            'UT Date':         header['UT-DATE'],
            'UT Time (start)': header['UT-TIME'],
            'UT Time (end)':   header['UT-END'],
            'Exposure (s)':    header['EXPTIME'],
            'Object':          header['OBJECT'],
            'Exposure type':   header['EXPTYPE'],
            'RA':              header['RA'],
            'Dec':             header['DEC'],
            'Mask':            header['SLITMASK'],
            'Filter':          header['FILTER'],
            'Disperser':       header['DISPERSR'],
            'Airmass':         header['AIRMASS'],
            'Seeing':          header['G-SEEING'],
            'Binning':         header['Binning'],
            'Speed':           header['Speed'],
            'Subrastr':           header['SUBRASTR'],
            #'File':            header['FILENAME'],
        }
    
df_file_info = pd.DataFrame.from_dict(file_info, orient='index')
df_file_info.index.name = 'root'
#sci_mask = df_file_info['Object'].str.contains('science')
#df_file_info[sci_mask]#.info()
#date = data_dir.split('/')[-1]
df_file_info.to_csv(f'/home/mango/Projects/HATP26b/night_log_{date}.csv', index=False)
#df_file_info.to_csv(f'./projects/HATP26b/night_log_{date}.txt', index=False)
#df_file_info.query('Object.str.contains("sci")')

KeyboardInterrupt: 

In [ ]:
df_file_info

In [ ]:
fpath = f'projects/HATP26b/journal/night_log_{date}.csv'
pd.read_csv(fpath)
df_sci = df_file_info.query('Object.str.contains("sci")')
df_cal = df_file_info.query('Object.str.contains("He")')

In [ ]:
#df_sci.nunique() #[cols].unique()
df_cal#.nunique() #[cols].unique()

In [ ]:
df_sci.tail()

In [ ]:
AM = df_sci['Airmass'].to_numpy()
idxs = range(len(AM))
idx_min = np.argmin(AM)

def plot_ann(ax, x, y):
    ax.plot(x, y, 'ro')
    ax.annotate(f'{y:.3f}', xy=(x, y+0.02))

fig, ax = plt.subplots()

ax.plot(idxs, AM)
plot_ann(ax, idxs[0], AM[0])
plot_ann(ax, idxs[idx_min], np.min(AM))
plot_ann(ax, idxs[-1], AM[-1])

ax.set_xlabel('Index')
ax.set_ylabel('Airmass')
ax.set_title('HAT-P-26 science frames')

utils.savefig('projects/HATP26b/journal/figures/data_inspection/airmass.pdf')

In [ ]:
data_dir = 'data/HATP23'
data_dict = {
    'transit_1':
        {
            'path':f'{data_dir}/ut160621',
            'ift':'1116',
            'sky_ap':20,
        },
#
    'transit_2':
        {
            'path':f'{data_dir}/ut170609',
            'ift':'0183',
            'sky_ap':25,
        },
    'transit_3':
        {
            'path':f'{data_dir}/ut180603',
            'ift':'0076',
            'sky_ap':25,
        },
    'transit_4':
        {
            'path':f'{data_dir}/ut180620',
            'ift':'4444',
            #'ift':'4050',
            'sky_ap':25,
        },
    'transit_5':
        {
            'path':f'{data_dir}/ut180821',
            'ift':'0200',
            #'ift':'0000',
            'sky_ap':25,
        },
}

In [6]:
data_dir = 'data/HATP26'
data_dict = {
    'transit_1':
    {
        'path':f'{data_dir}/ut190313',
        'ift':'1310',
        'sky_ap':25,
    }
}

In [10]:
for transit, transit_info in data_dict.items():
    dirpath = transit_info['path']
    fname = f"ift{transit_info['ift']}"
    sky_ap = transit_info['sky_ap']
    fig, im = utils.plot_chips(
        dirpath, fname, vmin=0, vmax=2_000, sky_ap=25, spec_ap=12,
    )
    #fig.set_size_inches(8, 11)
    #fig.colorbar(im, ax=axes.flatten(), aspect=30, label='counts')
    #fig.subplots_adjust(wspace=0.1, hspace=0.01)
    #plt.savefig("/Users/mango/Desktop/test.png", bbox_inches="tight")
    plt.savefig(
       f'projects/HATP26b/journal/figures/{transit}_raw_frame.png',
        bbox_inches="tight",
        dpi=250
    )

Traceback (most recent call last):
  File "/home/mango/miniconda3/envs/gen/lib/python3.8/site-packages/matplotlib/cbook/__init__.py", line 224, in process
    func(*args, **kwargs)
  File "/home/mango/miniconda3/envs/gen/lib/python3.8/site-packages/mpl_toolkits/axes_grid1/colorbar.py", line 715, in update_normal
    self.draw_all()
AttributeError: 'Colorbar' object has no attribute 'draw_all'


In [ ]:
axes.flatten()

In [ ]:
fig, axes = plt.subplots(2, 4, sharex=True, sharey=True, figsize=(8, 11))

for i, ax in enumerate(axes.flatten()):
    im = ax.imshow(np.random.rand(2048, 1024))
    if i <= 3:
        ax.set_title('title')
    else:
        ax.set_xlabel('title')
    
#cbar_ax = fig.add_axes([0.91, 0, 0.05, 0.7])
#fig.colorbar(im, cax=cbar_ax)
#fig.colorbar(im, ax=axes.ravel().tolist(), aspect=30, label='counts')
fig.subplots_adjust(wspace=0.1, hspace=0.01)

In [ ]:
sns.palplot(sns.color_palette())

### Chips Movie

In [ ]:
#plt.switch_backend("Agg")
dirpath = "./data/HATP26"
fnames = df_sci.index.values
for fname in tqdm(fnames):
    fig, axes = utils.plot_chips(dirpath, fname, vmin=0, vmax=1_000, sky_ap=25)
    fpath = f"{dirpath}/pngs/sci_mask_{fname}.png"
    fig.savefig(fpath, bbox_inches="tight")
    plt.close(fig)

### Chips Movie all dates

In [ ]:
plt.switch_backend("Agg")
frames_all_nights = []
for night in sorted(glob.glob("./data/WASP39/ut*")):
    frame_date = sorted(glob.glob(f"{night}/pngs/*.png"))
    frames_all_nights.append(frame_date)

# Get longest night for padding
max_night_length = len(max(frames_all_nights, key=len))

# Pad shorter nights with last frame for each night
for night in frames_all_nights:
    night += [night[-1]]*(max_night_length - len(night))

# Combine
#for frames_all in zip(*frames_all_nights):
#    for frame in frames_all:
#        print(frame)
    #print("\nMoving on to next frame in all datasets")

In [ ]:
plt.switch_backend("Agg")
for i, frame in tqdm(enumerate(zip(*frames_all_nights)), total=max_night_length):
    # Create plt plot:
    fig, axes = plt.subplots(2, 5, figsize=(40, 18))

    for ax, img in zip(axes.flat, frame):
        plt_image = np.array(Image.open(img))
        ax.imshow(plt_image, aspect="auto")
        ax.axis("off")

    fig.tight_layout()
    plt.subplots_adjust(wspace=.0, hspace=.0)
    plt.savefig(f"../atmospheres_meeting/figs/chips/frames/frame_{i:03}.png", 
                bbox_inches="tight", pad_inches=0)
    plt.close()

In [ ]:
import itertools

In [ ]:
frames_all_nights = []
for night in sorted(glob.glob("./data/HATP23/ut*")):
    frame_date = sorted(glob.glob(f"{night}/pngs/*.png"))
    frames_all_nights.append(frame_date)

In [ ]:
frames_all_nights = []
for night in sorted(glob.glob("./data/HATP23/ut*")):
    frame_date = sorted(glob.glob(f"{night}/pngs/*.png"))
    frames_all_nights.append(frame_date)

# Get longest night for padding
max_night_length = len(max(frames_all_nights, key=len))

# Pad shorter nights with last frame for each night
for night in frames_all_nights:
    night += [night[-1]]*(max_night_length - len(night))

# Combine
#for frames_all in zip(*frames_all_nights):
#    print(frames_all)
    #for frame in frames_all:
    #    print(frame)
    
    #print("\nMoving on to next frame in all datasets")

## Aperture

In [ ]:
fpath = "./data/HATP26/ut190313/ift1310c2.fits"
fig, axes, data = utils.plot_aperture(fpath)

## 2D Spectra over time

In [ ]:
spectra = []
for fpath in fpaths_sci:
    chip_data = utils.fits_data(fpath)
    x_l, x_r = 190, 206
    #x_l, x_r = 688, 691
    y_d, y_u = 0, chip_data.shape[0]
    spectra_n = chip_data[y_d:y_u, x_l:x_r]
    #spectra_n = np.max(spectra_n, axis=1)
    spectra.append(spectra_n)
    
spectra = np.concatenate(np.array(spectra), axis=1).T

In [ ]:
data_dir = "../data/WASP39/ut190315"
chip_num = 5
x_l, x_r = 190, 206
object_name = "w39bs science"

#data_dir = "../data/WASP43/ut180603"
#chip_num = 8
#x_l, x_r = 688, 691
#object_name = "science"

fig, ax, spectra = utils.plot_spec2d(data_dir, chip_num, x_l, x_r, object_name)

tokens = data_dir.split("/")
target, date = tokens[2], tokens[3]
fname = f"spectra_2D_{target}_{date}"
fig.savefig(f"/Users/mango/Desktop/{fname}.pdf", bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(spectra, vmin=0, vmax=30_000)
fig.colorbar(im, ax=ax)
ax.set_aspect("equal")
#fig.savefig(f"/Users/mango/Desktop/{fname}.pdf", bbox_inches="tight")

## Quicklook Light Curves

In [ ]:
data['fluxes'].keys()

In [ ]:
fpath = "./data/HATP23/ut180821/HATP23_WLC_OTG.pkl"
target = fpath.split('/')[2]
data = utils.pkl_load(fpath)
name_target = [s for s in data["fluxes"].keys() if target in s][0]

fig, ax = plt.subplots(figsize=FIG_WIDE)

# Plot divided LCs
time = data["time"]
flux_target = data["fluxes"][name_target]
flux_comps = []
for obj, flux in data["fluxes"].items():
    if obj != name_target :
        flux_comps.append(flux)
        flux = flux_target / flux
        #ax.plot(time, flux/np.median(flux), '.', label=obj)
        ax.plot(flux/np.median(flux), 'o', label=obj)
        
# Plot sum LC
flux_sum_comps = np.sum(flux_comps, axis=0)
flux = flux_target / flux_sum_comps
flux_norm = flux / np.median(flux)
#ax.plot(time, flux / np.median(flux), 'lightgrey', lw=5, label="all")
ax.plot(flux_norm, '-o', color='lightgrey', lw=5, label="all")
ax.legend(ncol=int((len(flux_comps)+1)/2), frameon=True)
delta = 1.0 - np.min(flux_norm)
RpRs = np.sqrt(delta)
label = rf"$R_\mathrm{{p}}/R_\mathrm{{s}} \approx {RpRs:.3f}$"
ax.annotate(label, xy=(0.9, 0.1), xycoords="axes fraction", ha="center")
ax.set_xlabel("Time (UTC)")
ax.set_ylabel("Normalized Flux")
ax.set_ylim(0.978, 1.04)
fig.tight_layout()

## IMACS Photometry

In [ ]:
data_dict = {
    'Transit 2':
    f'{data_dir}/ut170609_a15_25_noflat/LCs_hp23_{binsize}.pkl',

    'Transit 3':
    f'{data_dir}/ut180603_a15_25_noflat/LCs_hp23_{binsize}.pkl',
}

fig, axes = plt.subplots(2, 1, figsize=FIG_LARGE)

objs = ["HATP23b", "comp4", 'comp5']
colors = ["C5", "C0", "C1"]
for ax, (transit_name, transit_path) in zip(axes.flat, data_dict.items()):
    data = utils.pkl_load(transit_path)
    time = Time(data["t"], format="jd")# - 2450000
    wavs = data["optimal spectra"]["wavelengths"]
    wav_low_idx = np.where(wavs == 5000)[0][0]
    wav_high_idx = np.where(wavs == 9000)[0][0]
    wavs_int = wavs[wav_low_idx:wav_high_idx+1]
    for obj, c in zip(objs, colors):
        f = data["optimal spectra"][obj][:,wav_low_idx:wav_high_idx+1].sum(axis=1)
        ax.plot_date(time.plot_date, f/np.max(f), '.', label=obj, color=c)

        ax.legend(ncol=3, loc=4)
        ax.set_xlabel("Time (UTC)")
        ax.set_ylabel("$F / F_\mathrm{max}$")
        ax.set_title(transit_name)

    #ax.plot_date(time.plot_date, data["oLC"]/np.max(data["oLC"]), c='r')
fig.tight_layout()

In [ ]:
data.keys()

In [ ]:
plt.figure()
plt.plot(data["oLC"][:, 1], '.')

# ASAS-SN Photometry

#### Raw data

In [ ]:
set_theme("paper")
fpaths = [
    'projects/HATP23b/asas-sn_hatp23.csv',
    'projects/HATP23b/asas-sn_comp5.csv',
    'projects/HATP23b/asas-sn_comp4.csv',
    'projects/HATP23b/asas-sn_comp7.csv'
]

fig, ax = plt.subplots(figsize=FIG_WIDE)
for fpath in fpaths:
    df = pd.read_csv(fpath, parse_dates=["UT Date"], date_parser=utils.myparser)

    #display(df.head())
    name = fpath.split('/')[-1]
    df.set_index(["UT Date"], inplace=True)
    grouped = df.groupby('Filter')
    for (k, g) in grouped:
        if k=='V':
            ax.plot(g['flux(mJy)'], marker='.', lw=0, label=name)
    
mid_transit_times = {
        'Transit 1': '2016-06-22 08:18:00',
        'Transit 2': '2017-06-10 07:05:00',
        'Transit 3': '2018-06-04 07:24:00',
        'Transit 4': '2018-06-21 06:56',
        'Transit 5': '2018-08-22 03:30:00',
    }
p_kwargs = {'ls': '--', 'c': 'grey', 'lw':1.0}
for transit_name, t0 in mid_transit_times.items():
    t_mid = parser.parse(t0)
    ax.axvline(t_mid, **p_kwargs)
    ax.text(t_mid, 20.0, transit_name, ha='right', rotation=90.0)

ax.legend(loc=2, frameon=True, fontsize=12, bbox_to_anchor=(1, 1))
#ax.set_title(fpath)
ax.set_ylim(0, 60)
ax.set_xlabel('Date')
ax.set_ylabel('Flux (mJy)')
fig.autofmt_xdate(rotation=45)
fig.tight_layout()

In [ ]:
def reject_outliers(data, m=2):
    outlier_mask = abs(data - np.mean(data)) > m * np.std(data)
    return data[~outlier_mask], outlier_mask

In [ ]:
fpath = "projects/HATP23b/asas-sn_hatp23.csv"
df = pd.read_csv(fpath, parse_dates=["UT Date"], date_parser=utils.myparser)
df_V = df.groupby("Filter").get_group("V")
t = (df_V["HJD"] - 2.457e6).values
f = df_V["flux(mJy)"].values
plt.figure()
plt.plot(t[142:], '.')

In [ ]:
f_filtered, outlier_mask = reject_outliers(f)

fig, axes = plt.subplots(2, 1, figsize=FIG_LARGE)
ax_top, ax_bottom = axes

#ax.plot(t, f, '.')
#ax.plot(t[outlier_mask], f[outlier_mask], 'ro', alpha=0.5)

idxs_dict = {
    "asas-sn_hatp23.csv":{
        "idx_start":54,
        "idxs_seasons":
            [slice(0, 203), slice(204, 377), slice(412, 516)]
    },
    "asas-sn_comp4.csv":{
        "idx_start":7,
        "idxs_seasons":
            [slice(0, 203), slice(204, 377), slice(419, 531)]
    },
    "asas-sn_comp5.csv":{
        "idx_start":7,
        "idxs_seasons":
            [slice(0, 203), slice(204, 371), slice(418, 530)]
    },
    "asas-sn_comp7.csv":{
        "idx_start":142,
        "idxs_seasons":
            [slice(0, 202), slice(204, 377), slice(419, 531)]
    },
}

target_file = fpath.split('/')[-1]
idx_start = idxs_dict[target_file]["idx_start"]
time_global, flux_global = t[~outlier_mask][idx_start:], f[~outlier_mask][idx_start:]

idxs_seasons = idxs_dict[target_file]["idxs_seasons"]

ax_top.plot(time_global, flux_global, 'ko', alpha=0.25, mew=0)
# Compute the GLS periodogram with default options.
clp = pyPeriod.Gls((time_global, flux_global))
freq_max, freq_max_err = clp.hpstat["fbest"], clp.hpstat["f_err"]
P_max, P_max_err = 1.0/freq_max, clp.hpstat["Psin_err"]
label = f"$P_\mathrm{{max}} = {P_max:.3f} \pm {P_max_err:.3e}$"
ax_bottom.semilogx(1.0/clp.freq, clp.power, label=label)
print(f"> Global")
print(f"{fpath}")
clp.info()
print()

# Calculate sine wave associated with 'best-fit' frequency
bestSine_global = clp.sinmod(time_global)
ax_top.plot(time_global, bestSine_global, label="Global")
ax_top.legend(fontsize=12)

for i, idxs_season in enumerate(idxs_seasons, start=1):
    time, flux = time_global[idxs_season], flux_global[idxs_season]
    #ax.plot(time, flux, '.')

    # Compute the GLS periodogram with default options.
    clp = pyPeriod.Gls((time, flux))
    
    # and plot power vs. period.
    freq_max, freq_max_err = clp.hpstat["fbest"], clp.hpstat["f_err"]
    P_max, P_max_err = 1.0/freq_max, clp.hpstat["Psin_err"]
    label = f"$P_\mathrm{{max}} = {P_max:.3f} \pm {P_max_err:.3e}$"
    ax_bottom.semilogx(1.0/clp.freq, clp.power, label=label)
    ax_bottom.set_xlabel("Period (days)")
    ax_bottom.set_ylabel("Power")
    
    # Calculate sine wave associated with 'best-fit' frequency
    bestSine = clp.sinmod(time)
    ax_top.plot(time, bestSine, label=f"Season {i}")
    idxs_mid = len(time) // 2
    phot_var = np.var(flux)
    ax_top.annotate(
        f"$\sigma^2 = {phot_var:.3f}$",
        (time[idxs_mid], 41.5),
        ha="center",
        va="bottom",
    )

    # Print helpful information to screen
    print(f"> Season {i}")
    print(f"{fpath}")
    clp.info()
    print()
    
ax_top.set_xlabel("HJD - 2.457e6")
ax_top.set_ylabel("Flux (mJy)")
ax_top.set_title(fpath)
ax_top.legend(ncol=4, fontsize=12)
ax_bottom.legend(ncol=2, fontsize=12)
fig.tight_layout()

#utils.savepng(f"/Users/mango/Desktop/{target_file.split('.')[0]}.png")

#### GP Analysis

In [ ]:
set_theme("paper")
dirpath = 'projects/HATP23b/data/stell_act'

# Load processed data
df_stell_data = pd.read_csv(
    f'{dirpath}/HATP23_lc_norm_v3.csv',
    names=['t_HJD', 't_UT', 'f'],
    parse_dates=[1],
    infer_datetime_format=True,
)
#df_stell_data.set_index(["t_ut"], inplace=True)

# Load model data
df_stell_model = pd.read_csv(
    f'{dirpath}/HATP23_GP_model_Prot7_v3.csv',
    names=['t_HJD', 'f', 'f_err'],
)

print("data")
display(df_stell_data.head())
print("model")
display(df_stell_model.head())

# Load model

fig, ax = plt.subplots(figsize=FIG_WIDE)

#ax.plot(df_stell_data['f'], '.')
ax.plot(df_stell_data['t_HJD'], df_stell_data['f'], 'r.', alpha=0.5, mew=0)
ax.plot(df_stell_model['t_HJD'], df_stell_model['f'], color="grey")
f_d = df_stell_model['f'] - df_stell_model['f_err']
f_u = df_stell_model['f'] + df_stell_model['f_err']
ax.fill_between(df_stell_model['t_HJD'], f_d, f_u, alpha=0.3, lw=0, color="grey")

p_kwargs = {'ls': '--', 'c': 'darkgrey', 'lw':1.0}
trans = transforms.blended_transform_factory(ax.transData, ax.transAxes)

for transit_name, t0 in mid_transit_times.items():
    t_mid = Time(t0).jd - 2.4e6
    ax.axvline(t_mid, **p_kwargs)
    ax.annotate(
        transit_name,
        xy=(t_mid, 0.1),
        xycoords=trans,
        ha='right',
        rotation=90.0,
        fontsize=12,
    )

#ax.set_title(fpath)
#ax.set_xlim(57400, 58500)
ax.set_ylim(0.88, 0.98)
ax.set_xlabel('Date (HJD - 2400000)')
ax.set_ylabel('Flux (mJy)')
fig.tight_layout()

#utils.savefig("projects/HATP23b/paper/figures/phot_mon_full.pdf")

In [ ]:
fig, ax = plt.subplots()
ax.plot(df_stell_model['f'])
ax.axhline(np.median(df_stell_model['f']))

In [ ]:
(np.max(df_stell_model['f']) - np.median(df_stell_model['f'])) * 100

In [ ]:
Time(mid_transit_times["Transit 1"]).jd - 2400000

In [ ]:
(0.947726 - 0.93211)

In [ ]:
(13571.18577 - 12584.79890) / 12584.79890

In [ ]:
utils.weighted_mean_uneven_errors()

In [ ]:
(12585 + 12716 + 13571 + 13204 + 12832) / 5

In [ ]:
12585 - 12981.6

In [ ]:
f = 0.05
100*f / (1 - f)

In [ ]:
d = data #np.ones((8192, 1))
f=open("pyt2.pickle","wb")
pickle.dump(d, f)
f.close()

In [ ]:
data = utils.pkl_load("data/data_reductions/HATP23/ut180603_a15_25_noflat/LCs_hp23_species.pkl")

In [ ]:
df = pd.DataFrame(d.T)

In [ ]:
data["cLC"].shape

In [ ]:
data["spectra"]["HATP23b"].shape

In [ ]:
data.keys()

# Reduced data

## Traces over time

In [ ]:
dirpath = "./data/data_reductions/HATP26/ut190313_a15_25_noflat_LBR"
fig, axes, XX, YY = utils.plot_traces(dirpath)
#utils.savepng('projects/HATP23/traces_ut180821')

In [ ]:
len(XX["comp7_3"][0])

### Some typin

## Sub Images

In [ ]:
fpath = './data/data_reductions/HATP23/ut180603_a15_25_noflat/sub_images/ift0136_HATP23b_c5.fits'
data = utils.fits_data(fpath)

In [ ]:
fig, ax = plt.subplots()

ax.imshow(data)

In [ ]:
dir_sub = "../data/data_reductions/WASP39/ut190315_a8_18_noflat_LBR/sub_images"
fpath_subimg = f"{dir_sub}/ift3058_WASP39_c5.fits"
fpath_subtrace = f"{dir_sub}/trace_ift3058_WASP39_c5_ap_109.fits"

#dir_sub = "../data/data_reductions/WASP43/ut180603_a9_24_noflat_LBR/sub_images"
#fpath_subimg = f"{dir_sub}/ift0107_WASP43b_c8.fits"
#fpath_subtrace = f"{dir_sub}/trace_ift0107_WASP43b_c8_ap_25.fits"

subimg = utils.fits_data(fpath_subimg)
subtrace = utils.fits_data(fpath_subtrace)
print(f"subarray dimension: {subimg.shape}")
fig, axes, data = utils.plot_subaperture(fpath_subimg)
plt.show()

## Raw Extracted Spectra

In [12]:
fig, ax = plt.subplots(figsize=(12, 5))
dirpath = "./data/data_reductions/HATP26/ut190313_a15_25_noflat_LBR"
#dirpath = './data/data_reductions/HATP23_py2/ut180620_a15_25_noflat'
fpaths = np.array(glob.glob(f"{dirpath}/*spec.fits"))
#mask = np.array(["comp5_5" not in fpath for fpath in fpaths])
#mask &= ["comp7_3" not in fpath for fpath in fpaths]
#mask &= ["comp7_8" not in fpath for fpath in fpaths]

#mask = np.array(["c05_4" not in fpath for fpath in fpaths])
#mask &= ["c05_7" not in fpath for fpath in fpaths]

#mask = np.array(["com03_6" not in fpath for fpath in fpaths])
#fpaths = fpaths[mask]
objs = {}
for i, obj_path in enumerate(fpaths):
    p, p_name, wavs, p_data = utils.plot_spec_file_objects(
        ax, obj_path, i=1,
    )
    objs[p_name] = p_data

ax.set_title(dirpath)
ax.legend(loc=2, fontsize=12)
ax.set_xlim(0, 2_000)
#ax.set_ylim(-10, 90_000)
ax.set_xlabel("pixel (dispersion direction)")
ax.set_ylabel("counts")
fig.tight_layout()
#date = Path(dirpath).parts[-1].split('_')[0]
utils.savepng("/home/mango/Projects/HATP26b/journal/figures/spectra")

/home/mango/miniconda3/envs/gen/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1113: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
/home/mango/miniconda3/envs/gen/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [ ]:
fpath = 'data/data_reductions/HATP23/ut180821_a15_25_noflat/HATP23b_5_spec.fits'
data = utils.fits_data(fpath)

In [ ]:
plt.plot(data[10, 1, :])

## Wavelength Calibration

Some of the lamp spectra through the target and comparison star slits are usually completely shifted off of the reference lamp spectra, so `guess_lines.py` can't be used. Instead of using this, the lines can be manually identified from [NOAO](http://iraf.noao.edu/specatlas/henear/henear.html). After getting the first target done, the rest can be bootstrapped relatively quickly since their arc spectra should be similar.

To make things a little easier, the following routine will automatically record the pixel and wavelength coordinate of each line selected from the bottom panel. To select the pixel value under the mouse, press 'X' on the keyboard. To record the corresponding wavelength value, click on the annotated value in the top reference panel. Rinse and repeat.

### Select lines

In [ ]:
# Reference (NOAO), do this one first
# dirpath = "./useful_scripts"
# fpath_arc_ref = f"{dirpath}/noao_flux.fits"
# fpath_lines_ref = f"{dirpath}/noao_line_list.txt"

# Reference (target) arc spectra, use as ref after calibrating inital with NOAO
dirpath_ref = "./data/data_reductions/HATP26/ut190313_a15_25_noflat_LBR/arcs"
fpath_arc_ref = f"{dirpath_ref}/HATP26_2_arc.fits"
name = fpath_arc_ref.split('/')[-1].split('_')[0]
#fpath_lines_ref = f"{dirpath_ref}/{name}_guesses.txt"
fpath_lines_ref = f"{dirpath_ref}/{name}_lines_chips.csv"

# Arc spectra to compare
dirpath = "./data/data_reductions/HATP26/ut190313_a15_25_noflat_LBR/arcs"
fpath_arc = f"{dirpath}/c05_4_arc.fits"
name = fpath_arc.split('/')[-1].split('_')[0]
#fpath_lines = f"{dirpath}/{name}_guesses.csv"
fpath_lines = f"{dirpath}/{name}_lines_chips.csv"

# Plot
wavs, pixels = utils.compare_arc_lines(
    fpath_arc_ref=fpath_arc_ref, 
    fpath_lines_ref=fpath_lines_ref,
    fpath_arc=fpath_arc,
    fpath_lines=fpath_lines,
    sharex=False, sharey=False,
)

### Write to file

In [ ]:
df_guesses = pd.read_csv(fpath_lines, escapechar='#')
# update if lines chosen
#if len(wavs) != 0 and len(pixels) != 0:
chips = [int(fpath_arc.split('/')[-1].split('_')[1])] * len(wavs)
df_chosen = pd.DataFrame({"Wav":wavs, "Pix":pixels, "Chip":chips})

df_guesses = pd.concat([df_guesses, df_chosen])

fname = fpath_lines.split('/')[-1]
df_guesses = df_guesses.sort_values(by=["Chip", "Wav"])
display(df_guesses)
print(f"Will save to: {dirpath}/{fname}")

save = input("Continue? (y/n): ")
if save.lower() == 'y': 
    df_guesses.to_csv(f"{dirpath}/{fname}", index=False, escapechar='#')
    print(f"Saved to: {dirpath}/{fname}")
else: 
    print("not saved") 

In [ ]:
np.loadtxt(fpath, delimiter=',').shape

### View wavelength solution

In [109]:
# Plot
fig, ax = plt.subplots(figsize=FIG_LARGE)

dir_lines = "./data/data_reductions/HATP26/ut190313_a15_25_noflat_LBR/arcs"
#fpaths = glob.glob(f"{dir_lines}/*guesses*.txt")
fpaths = glob.glob(f"{dir_lines}/*lines_chips.csv")
x_name = "Pix"
y_name = "Wav"
for fpath in fpaths:
    df = pd.read_csv(fpath, escapechar="#")
    comp_name = fpath.split('/')[-1].split('_')[0].split("comp")[-1]
    utils.plot_pix_wav(ax, df, x_name, y_name, comp_name)

ax.legend(frameon=True)
ax.set_xlabel(x_name)
ax.set_ylabel(y_name)

fig.tight_layout()

## Final spectra

In [23]:
data_dir = "/home/mango/ACCESS_notebook/Projects/HATP26b/data_reductions"
binsize = "hp26_bins"
# data_dict = {
#     'Transit 1':
#     f'{data_dir}/ut160621_a15_20_noflat/LCs_hp23_{binsize}_160621.pkl',
    
#     'Transit 2':
#     f'{data_dir}/ut170609_a15_25_noflat/LCs_hp23_{binsize}.pkl',

#     'Transit 3':
#     f'{data_dir}/ut180603_a15_25_noflat/LCs_hp23_{binsize}.pkl',

#     'Transit 4':
#     f'{data_dir}/ut180620_a15_25_noflat/LCs_hp23_{binsize}.pkl',

#     'Transit 5':
#     f'{data_dir}/ut180821_a15_25_noflat/LCs_hp23_{binsize}.pkl',
# }

data_dict = {
    "Transit 1":
    f"{data_dir}/ut190313_a15_25_noflat_LBR/LCs_{binsize}.pkl"
}

#set_theme('paper')
#paper_path = 'projects/HATP23b/paper/figures'
wbins = pd.read_table(
    f"{data_dir}/hp26_bins.dat",
    names=['wav_d', 'wav_u'],
    skiprows=1,
    sep='\s+',
    comment='#',
)
for transit, fpath in data_dict.items():
    data = utils.pkl_load(fpath)
    spec = data['optimal spectra']
    wavs = spec["wavelengths"]
    print(wavs[[data['idx_usable_wl']]])
    
    fig, ax = plt.subplots(figsize=FIG_WIDE)
    c = 'darkgrey'
    for name, data in sorted(spec.items()):
        if name in 'HATP23b':
            median_kwargs = {'c':'C5'} 
        else:
            median_kwargs = None #{'c':c}
            c = 'grey'
        if (name != 'wavelengths'):
            p, wav, flux = utils._plot_spec_file(
                ax, data=data, wavs=wavs, label=name, median_kwargs=median_kwargs,
            )
    ax.legend(loc=1, fontsize=12)
    
    # bins
    for i, (w_d, w_u) in enumerate(zip(wbins['wav_d'], wbins['wav_u'])):
        c = 'k' if i % 2 == 0 else 'darkgrey'
        ax.axvspan(w_d, w_u, alpha=0.25, color=c, lw=0)
    
    # species lines
    species = {
        'Na I-D':5892.9, 
        'K I_avg':7682.0, 
        'Na I-8200_avg':8189.0
    }
    [ax.axvline(wav, ls='--', lw=1, color='grey') for name, wav in species.items()]
    title = transit
    ax.set_xlabel('Wavelength (Å)')
    ax.set_ylabel('Normalized flux')
    #ax.set_title(title)
    ax.set_xlim(wavs[0], wavs[-1])
    ax.set_ylim(-0.01, 1.15)

    title = title.lower().replace(' ', '_') + '_extr_spec'
    fig.set_size_inches(FIG_WIDE)
    #utils.savefig(f'{paper_path}/extracted_spectra/{title}.pdf')
    #utils.savepng("/home/mango/Projects/HATP26b/journal/figures/spectra")

[4791.25 4792.5  4793.75 ... 8831.25 8832.5  8833.75]


In [ ]:
data = utils.pkl_load(fpath)
data.keys()

In [ ]:
data_dict = {
    'Transit 2':
    f'{data_dir}/ut170609_a15_25_noflat/LCs_hp23_{binsize}.pkl',

    'Transit 3':
    f'{data_dir}/ut180603_a15_25_noflat/LCs_hp23_{binsize}.pkl',
}

fig, axes = plt.subplots(2, 1, figsize=FIG_LARGE)

objs = ["HATP23b", "comp4", 'comp5']
colors = ["C5", "C0", "C1"]
for ax, (transit_name, transit_path) in zip(axes.flat, data_dict.items()):
    data = utils.pkl_load(transit_path)
    time = Time(data["t"], format="jd")# - 2450000
    wavs = data["optimal spectra"]["wavelengths"]
    wav_low_idx = np.where(wavs == 5000)[0][0]
    wav_high_idx = np.where(wavs == 9000)[0][0]
    wavs_int = wavs[wav_low_idx:wav_high_idx+1]
    for obj, c in zip(objs, colors):
        f = data["optimal spectra"][obj][:,wav_low_idx:wav_high_idx+1].sum(axis=1)
        ax.plot_date(time.plot_date, f/np.max(f), '.', label=obj, color=c)

        ax.legend(ncol=3, loc=4)
        ax.set_xlabel("Wavelength (Å)")
        ax.set_ylabel("$F / F_\mathrm{max}$")

fig.tight_layout()

In [108]:
cNames

['c01', 'c03', 'c05']

In [67]:
wbins = pd.read_table('hp26_bins.dat', names=['wav_d', 'wav_u'], skiprows=1, sep='\s+')

In [68]:
fpath = 'data/data_reductions/HATP26/ut190313_a15_25_noflat_LBR/LCs_w43_25nm.pkl'
data = utils.pkl_load(fpath)
spec = data['optimal spectra']
wavs = spec["wavelengths"]
print(wavs[[data['idx_usable_wl']]])

fig, ax = plt.subplots(figsize=FIG_WIDE)
p, wav, flux = utils._plot_spec_file(ax, data=spec['HATP26'], wavs=wavs)

[4791.25 4792.5  4793.75 ... 8831.25 8832.5  8833.75]


<ipython-input-68-a7c3ec140109>:5: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  print(wavs[[data['idx_usable_wl']]])
/home/mango/miniconda3/envs/gen/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1113: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
/home/mango/miniconda3/envs/gen/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [73]:
dw = 200
for w in np.arange(8109, 9069+40, dw):
    print(f'{w:.2f}', f'{w+dw:.2f}')

8109.00 8309.00
8309.00 8509.00
8509.00 8709.00
8709.00 8909.00
8909.00 9109.00


In [76]:
pd.options.display.float_format = '{:.2f}'.format
wbins = pd.read_table('hp26_bins.dat', names=['wav_d', 'wav_u'], skiprows=1, sep='\s+', comment='#')

offs = 0
wbins['wav_d'] -= offs
wbins['wav_u'] -= offs

wbins['diff'] = wbins['wav_u'] - wbins['wav_d']
wbins['center'] = (wbins['wav_u'] + wbins['wav_d']) / 2
wbins.rename(
    columns={
        "wav_d": "Wav start",
        "wav_u": "Wav end",
        "diff": "Wav diff",
        "center": "Wav cen",
    },
#).to_csv('projects/HATP23b/data/wavbins/species.csv', index=False)
)#.to_clipboard(index=False)
wbins

,wav_d,wav_u,diff,center
0,4800.00,5000.00,200.00,4900.00
1,5000.00,5200.00,200.00,5100.00
2,5200.00,5400.00,200.00,5300.00
3,5400.00,5600.00,200.00,5500.00
4,5600.00,5800.00,200.00,5700.00
5,5800.00,5985.80,185.80,5892.90
6,5985.80,6185.80,200.00,6085.80
7,6185.80,6385.80,200.00,6285.80
8,6385.80,6655.00,269.20,6520.40
9,6655.00,6855.00,200.00,6755.00


In [ ]:
wav_d	wav_u
5775.799999999999	5830.0
5820.799999999999	5875.0
5865.799999999999	5920.0
5910.799999999999	5965.0
5955.799999999999	6010.0
7645.799999999999	7665.0
7655.799999999999	7675.0
7665.799999999999	7685.0
7675.799999999999	7695.0
7685.799999999999	7705.0
8080.799999999999	8130.0
8120.799999999999	8170.0
8160.799999999999	8210.0
8200.8	8250.0
8240.8	8290.0


In [ ]:
fig, ax = plt.subplots()

# spectrum
#x.plot(wav, flux)

# wav bins
"""
wbins = {}
wbins['wav_d'] = []
wbins['wav_u'] = []
dw = 200
for w in range(5300, 9000, dw):
    wbins['wav_d'].append(w)
    wbins['wav_u'].append(w+dw)
"""
print(len(wbins['wav_d']))
for i, (w_d, w_u) in enumerate(zip(wbins['wav_d'], wbins['wav_u'])):
    c = 'b' if i % 2 == 0 else 'r'
    ax.axvspan(w_d, w_u, alpha=0.25, color=c)

# species lines
species = {
    'Na I-D':5892.9, 
    'Hα':6564.6, 
    'K I_avg':7682.0, 
    'Na I-8200_avg':8189.0
}
[ax.axvline(wav, ls='--', color='w') for name, wav in species.items()]

plt.show()

In [ ]:
(5785-5300)/5

In [ ]:
(6440-6010)/5

In [ ]:
(7657 - 6690)

In [ ]:
fig, ax = plt.subplots(figsize=(11, 5))
for name, data in sorted(spec.items()):
    if name != 'wavelengths':
        p, _ = utils._plot_spec_file(ax, data=data, wavs=wavs, label=name)
        
ax.set_xlim(wavs[0], wavs[-1])
ax.set_ylim(-0.01, 1.15)
ax.legend(fontsize=12)
fig.tight_layout()

## Light Curves

### WLC

In [3]:
dirpath = "/home/mango/Projects/HATP26b/data_detrending/hp26_190313_c/"
# data_dict = {
#     "Transit 2":
#     { 
#         "fpath":f"{dirpath}/ut170609_a15_25_noflat/LCs_hp23_bins.pkl",
#         "idxs_oot":"0:28, 128:164",
#     },
#     "Transit 3":
#     { 
#         "fpath":f"{dirpath}/ut180603_a15_25_noflat/LCs_hp23_bins.pkl",
#         "idxs_oot":"0:39, 119:180",
#     },
#     "Transit 4":
#     { 
#         "fpath":f"{dirpath}/ut180620_a15_25_noflat/LCs_hp23_bins.pkl",
#         "idxs_oot":"0:19, 128:171",
#     },
# }

data_dict = {
    "Transit 1":
    {
        "fpath":f"{dirpath}/ut190313_a15_25_noflat_LBR/LCs_w43_25nm.pkl",
        "idxs_oot": "[]",
    }
}

for transit_name, transit_data in data_dict.items():
    fpath = transit_data["fpath"]
    data = utils.pkl_load(fpath)

    fig, axes = plt.subplots(1, 3, figsize=FIG_WIDE)
    ax_left, ax_middle, ax_right = axes

    # Raw fluxes
    p, _ = utils.plot_fluxes(ax_left, data, use_time=False)
    p.set_title("Raw fluxes")
    p.set_ylim(0, 3e8)

    # OOT raw fluxes
    idx_oot = utils._bad_idxs(transit_data["idxs_oot"])
    p, p_data = utils.plot_fluxes(ax_middle, data, oot=True, idx_oot=idx_oot)
    p.set_title("OOT raw fluxes")
    p.set_ylim(0, 3e8)

    # Divided OOT raw fluxes
    time = p_data["time"]
    targ_flux = p_data["targ_flux"]
    comp_fluxes = p_data["comp_fluxes"]
    cNames = p_data["cNames"]
    p = utils.plot_div_fluxes(
        ax_right, time, targ_flux, comp_fluxes, cNames,
    )
    p.set_title("Divided OOT raw fluxes")
    p.set_ylim(0.5, 2.0)
    #np.savetxt(f"/Users/mango/Desktop/{title}.txt", fluxes)

    title = transit_name
    fig.suptitle(title)
    fig.tight_layout()

    #utils.savepng(f"/Users/mango/Desktop/oot_{title}.png")

    f_data = {
        "time":p_data["time"],
        "targ":p_data["targ_flux"],
        "comp5":p_data["comp_fluxes"][:, 0],
        "comp4":p_data["comp_fluxes"][:, 1],
    }
    #pd.DataFrame(f_data).to_csv(f"/Users/mango/Desktop/fluxes_{title}.csv", index=False)

['c01', 'c03', 'c05']
['c01', 'c03', 'c05']


In [ ]:
pd.DataFrame(f_data)

### WLC Divided

In [5]:
data = utils.pkl_load(data_dict["Transit 1"]["fpath"])
data.keys()

dict_keys(['spectra', 'optimal spectra', 'Z', 'g', 'rot', 'fo', 't', 'ha', 'telev', 'theta', 'derg', 'drot', 'etimes', 'deltas', 'wbins', 'cNames', 'oLC', 'cLC', 'idx_usable_wl', 'oLCw', 'cLCw', 'traces', 'apertures', 'sky', 'fwhm'])

In [6]:
data["cNames"]

['c01', 'c03', 'c05']

In [7]:
#set_theme("paper")
#sns.set_context('paper')
data_dir = "/home/mango/ACCESS_notebook/Projects/HATP26b/data_reductions"
LCs_pickle = "LCs_hp26_bins_r.pkl"
data_dict = {
    "Transit 1":
    {
        "path":f"{data_dir}/ut190313_a15_25_noflat_LBR/{LCs_pickle}",
        "bad_idxs":"[4,5,11,12]",
        "t0":2455304.65218,
    }
}
# data_dict = {
#     'Transit 1':
#         {
#             'path':f'{data_dir}/ut160621_a15_20_noflat/{LCs_pickle}_160621.pkl',
#             'bad_idxs':'[239, 240, 241, 256, 276, 278, 283, 284, 288:299]',
#             't0':2457561.84167,
#         },
    
#     'Transit 2':
#         {
#             'path':f'{data_dir}/ut170609_a15_25_noflat/{LCs_pickle}.pkl',
#             'bad_idxs':'[10]',
#             't0':2457914.79167,
#         },
    
#     'Transit 3':
#         {
#             'path':f'{data_dir}/ut180603_a15_25_noflat/{LCs_pickle}.pkl',
#             'bad_idxs':'[0:17]',
#             't0':2458273.80556,
#         },
    
#     'Transit 4':
#         {
#             'path':f'{data_dir}/ut180620_a15_25_noflat/{LCs_pickle}.pkl',
#             'bad_idxs':'[93, 94, 149, 157:171]',
#             't0':2458290.78472,
#         },
#     'Transit 5':
#         {
#             'path':f'{data_dir}/ut180821_a15_25_noflat/{LCs_pickle}.pkl',
#             #'bad_idxs':'[4:6, 51, 53, 54, 56:59, 64, 66, 68, 69, 72, 77:80, 87:94, 96:98, 101, 108:119, 120, 129:131, 133, 174:181]',
#             'bad_idxs':'[4:6, 51, 53, 54, 56:59, 64, 66, 68, 69, 72, 77:80, 87:94, 96:98, 101, 129:131, 133, 174:181]',
#             #'bad_idxs':'[108:120]',
#             't0':2458352.64028,
#         },
    
# }

comps = ['c01', 'c03', 'c05']
ncomps = len(comps)
fig, axes = plt.subplots(
    1, ncomps, 
    sharex=True, sharey=True,
    #figsize=(7, 14),
)
if axes.ndim == 1: axes = axes.reshape(1, axes.size) 
colors = sns.color_palette()
for i, (transit_name, transit_info) in enumerate(data_dict.items()):
    fpath = transit_info['path']
    t0 = transit_info['t0']
    data = utils.pkl_load(fpath)
    print(fpath)
    print(len(data['t']))
    print(data['cNames'])
    #ncomps = len(data['cNames'])
    #fig, axes = plt.subplots(
    #    1, ncomps, 
    #    sharex=True, sharey=True,
    #    figsize=(12,3),
    #)
    
    for ax, comp in zip(axes[i, :], comps):
        p = utils.plot_divided_wlcs(
            ax,
            data,
            t0=t0,
            ferr=0.001,
            comps_to_use=[comp],
            bad_idxs_user = transit_info['bad_idxs'],
            div_kwargs={'fmt':'.', 'lw':0.5, 'mew':0.0, 'ms':3, 'c':colors[i]},
            bad_div_kwargs={'fmt':'.', 'lw':0.5, 'mec':'k', 'ms':6, 'c':'w'},
        )
        #ax.set_title(comp)
        ax.annotate(
            rf"{transit_name}/{comp}",
            xy=(0.05, 0.85),
            xycoords='axes fraction',
            fontsize=12, color=colors[i],
            weight='bold',
        )
        #ax.set_xlim(-3, 3)
        #ax.xaxis.set_ticks([-3, -2, -1, 0, 1, 2, 3])
        ax.set_ylim(0.988, 1.02)
        ax.yaxis.set_ticks([0.96, 0.98, 1.00, 1.02, 1.04])
        #fig.set_size_inches(12, 3)

    print()
    
    title = transit_name.lower().replace(' ', '_') + '_extr_wlcs'
# Custom legend
#handles, labels = [], []
#for transit_name in list(data_dict.keys()):
#    dummy = plt.errorbar([], [], yerr=0.1, fmt='o', ms=5, elinewidth=2, label=transit_name)
#    handles.append(dummy)
#    labels.append(transit_name)
#fig.legend(handles, labels, ncol=5, bbox_to_anchor=(0.54, 1.1), loc='upper center')#, ncol=len(data_dict)+1) 
#axes[0, 0].set_title('comp4')
#axes[0, 1].set_title('comp5')

fig.text(0.54, -0.02, 'Time from estimated mid-transit (hours)', ha='center')
fig.text(-0.01, 0.5, 'Normalizd flux', va='center', rotation='vertical')
#axes[0].set_ylabel('Normalized flux')
fig.tight_layout()
#fig.set_size_inches(12, 3)
#utils.savepng("/home/mango/Projects/HATP26b/journal/figures/wlc")

    #plt.savefig('/Users/mango/Desktop/wlc_py2_py3.png', dpi=250, bbox_inches='tight')

/home/mango/ACCESS_notebook/Projects/HATP26b/data_reductions/ut190313_a15_25_noflat_LBR/LCs_hp26_bins_r.pkl
271
['c01', 'c03', 'c05']
c01 1-sigma bad_idxs: [15]
c03 1-sigma bad_idxs: [12, 17]
c05 1-sigma bad_idxs: []



In [54]:
comps = ["a1", "a2", "a3"]
fig, axes = plt.subplots(3, 1)
if axes.ndim == 1: axes = axes.reshape(1, axes.size) 
plt.close()

for ax, comp in zip(axes[0, :], comps):
    print(ax, comp)

AxesSubplot(0.125,0.653529;0.775x0.226471) a1
AxesSubplot(0.125,0.381765;0.775x0.226471) a2
AxesSubplot(0.125,0.11;0.775x0.226471) a3


In [53]:
axes[1]

array([<AxesSubplot:>], dtype=object)

In [ ]:
import h5py

In [ ]:
np.savez

In [ ]:
print(sorted(set([6, 51, 52, 55, 59, 64, 65, 66, 67, 68, 69, 72, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 100, 101, 108, 111, 112, 113, 120, 129, 130, 131, 132, 133, 178, 179, 180]).union(set([55, 56, 59, 68, 88, 89, 90, 91, 97, 120, 178]))))

In [ ]:
fig, ax = plt.subplots(figsize=FIG_WIDE, sharex=True, sharey=True)

comps_to_use =  ["comp4", "comp5"] #data["cNames"]
p, i_used, t, f_div_sum, i_not_used, i_all = utils.plot_sum_divided_wlcs(
    ax,
    data,
    comps_to_use=comps_to_use,
    bad_idxs_user = "[53:59, 64, 65, 68, 78:80, 88:91, 97, 109, 110:120, 173:181]",
    div_sum_kwargs={"fmt":'.', "mew":0, "alpha":0.5, "lw":0.5},
)
p.legend(ncol=3, loc="upper center", frameon=True)

fig.tight_layout()

In [ ]:
# Preview divided LC
fig, ax = plt.subplots(figsize=FIG_WIDE)

ax.errorbar(i_used, f_div_sum[i_used], yerr=0.001, fmt='.', lw=0.5)

fig.tight_layout()

In [ ]:
ferr = 0.001 * np.ones_like(i_used)
instrument = ["MAGELLAN"] * len(i_used)
df_juliet = pd.DataFrame(
    zip(t[i_used] - 2457000, f_div_sum[i_used].flatten(), ferr, instrument),
)
df_juliet

In [ ]:
p = pathlib.Path("juliet/hatp23_test/")
p.mkdir(parents=True, exist_ok=True)
df_juliet.to_csv(
    f"{p}/lc.dat",
    float_format="%.10f",
    sep=' ',
    header=False,
    index=False,
)

with open(p / "used_comps_and_bad_idxs.txt", "w") as f:
    f.writelines([f"{comps_to_use}\n", f"{i_not_used}"])
    
print("Saved")

### Binned

In [27]:
data = utils.pkl_load(
    "/home/mango/ACCESS_notebook/Projects/HATP26b/data_reductions/ut190313_a15_25_noflat_LBR/LCs_hp26_bins_r.pkl"
)

# Get target flux
flux_target = np.c_[data["oLCw"]]

# Select comps
cNames = data["cNames"]
comps_to_use = ["c01", "c03", "c05"] #cNames
comps_to_use_idxs = [cNames.index(cName) for cName in comps_to_use]
flux_comps_used = np.c_[data["cLCw"][:, comps_to_use_idxs, :]]
flux_comps_used_sum = np.c_[np.sum(flux_comps_used, axis=1)]

# Divide target flux by sum of chosen comp sum
flux_div_sum = flux_target / flux_comps_used_sum
flux_div_sum /= np.median(flux_div_sum, axis=0)

# Identify bad time idxs
time = data['t']
bad_idxs = '[4,5,11,12]' #'[4:6, 51, 53, 54, 56:59, 64, 66, 68, 69, 72, 77:80, 87:94, 96:98, 101, 108:119, 120, 129:131, 133, 174:181]'
bad_idxs = utils._bad_idxs(bad_idxs)
if bad_idxs is not None:
    idxs_used = np.delete(range(len(time)), bad_idxs)
else:
    idxs_used = range(len(time))

fig, axes = plt.subplots(1, 2, figsize=(8, 11), sharex=True, sharey=True)
ax_left, ax_right = axes

fluxes = flux_div_sum[idxs_used, :]
N_bins = fluxes.shape[1]

# Plot binned light curves
offs = 0.008
p_left = utils.plot_binned(
        ax_left,
        idxs_used,
        fluxes,
        utc=False,
        bins=np.array(data["wbins"]), 
        offset=offs,
        colors=np.array(sns.color_palette("Spectral_r", N_bins)),
        plot_kwargs={"marker":'.', "mew":0, "lw":0}
    )
p_left.set_title("Raw divided flux")

# TODO: Try detrending with `exoplanet`
# Plot residuals
p_right = utils.plot_binned(
        ax_right,
        idxs_used,
        np.ones_like(fluxes),
        utc=False,
        bins=np.array(data["wbins"]), 
        offset=offs,
        colors=np.array(sns.color_palette("Spectral_r", N_bins)),
        annotate=True,
        annotate_kwargs={"fontsize":10, "ha":"center"},
    )
p_right.set_title("Residuals")

ax_left.set_ylabel("Normalized flux + offset")
fig.text(0.5, 0, 'Time (JD)', ha='left')

fig.tight_layout()

#plt.savefig("/Users/mango/Desktop/binned_lc.png", dpi=250, bbox_inches="tight")

#utils.savepng("/home/mango/Projects/HATP26b/journal/figures/raw_blcs")

In [ ]:
data['idx_usable_wl']

In [ ]:
data['optimal spectra']['wavelengths']#[data['idx_usable_wl']]

In [ ]:
data['idx_usable_wl'].shape

In [ ]:
# Save to file
data = np.c_[time[idxs_used], fluxes]
p = pathlib.Path("xo/WASP43/ut180603/binned_free_rho_star")
p.mkdir(parents=True, exist_ok=True)
np.save(f"{p}/lc_w.npy", data)

In [ ]:
np.load(p / "lc_w.npy").shape

# Detrending

## PCA

In [ ]:
# Get comp idxs
cNames = data["cNames"]
comps_to_use = ["comp4"]
idxs_comps_to_use = [cNames.index(cName) for cName in comps_to_use]

# Get time idxs
bad_idxs = utils._bad_idxs("[239:241, 256, 276, 296]")
t = data['t']
idxs = range(len(t))
idxs_to_use = np.delete(idxs, bad_idxs)

# Get target and comp fluxes (in magnitude-space)
target_flux = data["oLC"][idxs_to_use]
comp_fluxes = data["cLC"][np.ix_(idxs_to_use, idxs_comps_to_use)]

target_flux_mag = -2.51*np.log10(target_flux)
comp_fluxes_mag = -2.51*np.log10(comp_fluxes)

# Median-substracted target (tms) and comparison (cms) lightcurves (in magnitude-space)
# tms will also have the time in ints first column
tms = target_flux_mag - np.median(target_flux_mag, axis=0)
tms = np.c_[t[idxs_to_use], tms]
cms = comp_fluxes_mag - np.median(comp_fluxes_mag, axis=0)

# Mean-subtract and rms normalize
#X = eparams #(tms - np.mean(tms, axis=0)) / np.std(tms, axis=0)
Xc = (cms - np.mean(cms, axis=0)) / np.std(cms, axis=0)

PCA = False
if PCA:
    eigenvectors, eigenvalues, Xc = utils.classic_PCA(Xc) 

#plt.plot(X[:, 1], '.')
#plt.plot(Xc[:, 1], '.')
#plt.plot(np.array(idxs)[idxs_to_use], target_flux, '.')

In [ ]:
lc_params = {
    'ld_law': 'logarithmic',
    'q1': 0.5,
    'q2': 0.5,
    't0': 2457561.8458,
    'P': 1.2128867,
    'p': 0.1113,
    'a': 4.26,
    'inc': 85.10,
}

t = np.linspace(np.min(t), np.max(t), 100)
lc_model = utils.get_transit_model(t, lc_params)
plt.plot(t, lc_model)

## Juliet

In [ ]:
# External params

# Get median of FWHM, background flux, accross all wavelengths, and trace position of zero point.
# First, find chips-names of target:
target = b"HATP23"
names = []
for name in data['fwhm'].keys():
    if target in name:
        names.append(name)
if len(names) == 1:
    Xfwhm = data['fwhm'][names[0]]
    Xsky = data['sky'][names[0]]
else:
    Xfwhm = np.hstack((data['fwhm'][names[0]],data['fwhm'][names[1]]))
    Xsky = np.hstack((data['sky'][names[0]],data['sky'][names[1]]))

fwhm = np.zeros(Xfwhm.shape[0])
sky = np.zeros(Xfwhm.shape[0])
trace = np.zeros(Xfwhm.shape[0])
target = "HATP23"
for i in range(len(fwhm)):
    idx = np.where(Xfwhm[i,:]!=0)[0]
    fwhm[i] = np.median(Xfwhm[i,idx])
    idx = np.where(Xsky[i,:]!=0)[0]
    sky[i] = np.median(Xsky[i,idx])
    trace[i] = np.polyval(data['traces'][target][i],Xfwhm.shape[1]/2)
    
eparams = np.c_[data['t'] - 2457000, data['Z'], data["deltas"][f"{target}_final"], fwhm, sky, trace]
eparams = eparams[i_used]

In [ ]:
columns = ['Time', 'Airmass', 'Delta_Wav', 'FWHM', 'Sky_Flux', 'Trace_Center']
df_eparams = pd.DataFrame(eparams, columns=columns)

df_eparams["Instrument"] = ["MAGELLAN"] * df_eparams.shape[0]
df_eparams.to_csv(
    "juliet/hatp23_test/GP_lc_regressors.dat",
    sep=' ',
    index=False,
    header=False,
    float_format="%.10f",
)
df_eparams

## GPTS

### WLC

#### LC

In [5]:
#set_theme('paper')
# data = {
#     #'Transit 1':'160621',
#     #'Transit 2':'170609',
#     #'Transit 3':'180603',
#     #'Transit 4':'180620',
#     'Transit 5':'180821',
# }
data = {"Transit 1":"190313"}
PCA = 'PCA_2'

for i, (transit, date) in enumerate(data.items()):
    fig, axes = plt.subplots(
        2, 1,
        sharex=True,
        gridspec_kw={'height_ratios':[5, 1]}
    )
    dirpath = f"/home/mango/ACCESS_notebook/Projects/HATP26b/data_detrending/hp26_{date}_r/white-light"
    ax_top, ax_bottom = axes
    # get t0, P
    fpath_t0 = f'{dirpath}/results.dat'
    df_results = pd.read_table(
        fpath_t0, sep='\s+', index_col=0, escapechar='#',
    )
    t0, P = df_results.loc[['t0', 'P']]['Value']

    fpath = f"{dirpath}/{PCA}/detrended_lc.dat"
    
    # Detrended data
    t, detflux, detflux_err, model = np.genfromtxt(fpath, unpack=True)
    phase = utils.get_phases(t, P, t0)
    ax_top.plot(phase*24, detflux, 'o', label=transit,c=f"C{i}", mew=0) 
    c_dark = 0.5*np.array(mpl.colors.to_rgba(f'C{i}')[:3])
    
    # Full model
    t_full, f_full = np.genfromtxt(f'{dirpath}/full_model_{PCA}.dat').T
    phase_full = utils.get_phases(t_full, P, t0)
    p = ax_top.plot(phase_full*24, f_full, color=c_dark, lw=2, zorder=10)
    
    # Residuals
    c = 2 * c_dark
    resids = detflux - model
    ax_bottom.plot(phase*24, resids*1e6, '.', mew=0, color=c)
    trans = transforms.blended_transform_factory(
            ax_bottom.transData, ax_bottom.transAxes
        )
    ax_bottom.axhline(0, lw=2, c=c_dark)

    rms = np.std(resids*1e6)
    ax_bottom.annotate(
        f"{int(rms)}",
        xy=(1.02*phase[-1]*24, 0.55),
        xycoords=trans,
        fontsize=12,
    )

    #ax_top.legend(loc=4, frameon=True)
    #ax_top.set_title(title)
    #fig.text(0.5, 0, 'Phase (hours)', ha='left')
    ax_bottom.set_xlabel('Phase (hours)')
    #ax_top.set_xlim(-1., 1.)
    #ax_top.set_ylim(0.990, 1.002)
    #ax_bottom.set_ylim(-2000, 2000)
    ax_top.set_ylabel("Normalized flux")
    ax_bottom.set_ylabel('ppm')
    #ax.xaxis.set_major_locator(HOURS)
    #ax.set_xlim(2.457561e6 + 0.76, 2.457561e6 + 0.92)
    #ax.set_ylim(0.982, 1.00143)

    fig.set_size_inches(FIG_WIDE)
    fig.tight_layout()
    title = transit.lower().replace(' ', '_') + '_detr_wlcs'
    #utils.savefig(f'projects/HATP23b/paper/figures/detrended_wlcs/{title}.pdf')

    #plt.savefig("/Users/mango/Desktop/wlc_gpts_ut160621.png", dpi=250, bbox_inches="tight")
    #utils.savepng("/home/mango/Projects/HATP26b/journal/figures/det_lc")

In [ ]:
plt.plot(phase, detflux, '.')

In [ ]:
t, f = np.genfromtxt(f'{dirpath}/full_model_ut180821.dat').T

In [ ]:
for i in range(1, 3):
    print(i)

In [ ]:
pd.read_table(fpaths[0], sep='\s+', index_col=0).loc['p']

In [ ]:
dirpath = './data/data_detrending/HATP23'
fpaths = sorted(glob.glob(f'{dirpath}/hp*/white-light/results.dat'))

ps = []
for fpath in fpaths[1:]:
    print(fpath)
    p = pd.read_table(fpath, sep='\s+', index_col=0).loc['p'].to_numpy()
    ps.append(p)
ps = np.array(ps)

p_means, p_us, p_ds = ps.T

p_uncs=np.max([p_us, p_ds], axis=0)
np.average(p_means, weights=1/p_uncs**2)
#np.mean(p_means)

In [ ]:
ps

In [ ]:
p_means, p_us, p_ds = ps.T

p_uncs=np.max([p_us, p_ds], axis=0)
np.average(p_means, weights=1/p_uncs**2)
#np.mean(p_means)

In [ ]:
print(f'{p_means[0]:.9f}')

### Parameter Summary

In [ ]:
data_dir = 'data/data_detrending/HATP23_c'
binsize = 'custom'
data_dict = {
    'Transit 1':
    f'{data_dir}/hp23b_160621_{binsize}/white-light/results.dat',
    
    'Transit 2':
    f'{data_dir}/hp23b_170609_{binsize}/white-light/results.dat',
    
    'Transit 3':
    f'{data_dir}/hp23b_180603_{binsize}/white-light/results.dat',
    
    'Transit 4':
    f'{data_dir}/hp23b_180620_{binsize}/white-light/results.dat',
    
    'Transit 5':
    f'{data_dir}/hp23b_180821_{binsize}/white-light/results.dat',
}

# truths from Sada and Ramon (2016) + GAIA DR2
with open(f'{data_dir}/truth.json', 'rb') as f:
    parameters = json.load(f)

parameters['d'] = {
    'symbol': '$\delta$',
    'truth': [0.1113**2 * 1e6, 0.001**2 * 1e6, 0.0009**2 * 1e6],
    'definition': 'transit depth (ppm)'
}

# holds results from each transit
df_results = {}
for transit, fpath in data_dict.items():
    df_results[transit] = pd.read_table(fpath, sep='\s+',index_col='Variable')
    df_results[transit].loc['t0']['Value'] -= 2450000
    p_val, p_u, p_d = df_results[transit].loc['p'].values.T
    df_results[transit].loc['d'] = p_val**2 * 1e6, p_u**2 * 1e6, p_d**2 * 1e6
    
def write_latex(param, df):
    v, vu, vd = df.loc[param]
    return f'{v:.5f}^{{+{vu:.5f}}}_{{-{vd:.5f}}}'

# Create summary table of all transits
results_dict = {}
results_dict['parameter'] = [p["symbol"] for p in parameters.values()]
#results_dict['definition'] = [p["definition"] for p in parameters.values()]
for transit, results in df_results.items():
    results_dict[transit] = []
    for param, param_info in parameters.items():
        results_dict[transit].append(write_latex(param, results))
        
results_table = pd.DataFrame(results_dict)
results_table#.to_clipboard(index=False)

In [ ]:
df_results["Transit 1"].index

In [ ]:
utils.weighted_mean_uneven_errors(*get_params_and_uncs("p"))[0]**2*1e6

In [ ]:
utils.weighted_mean_uneven_errors(*get_params_and_uncs("d"))

In [ ]:
get_params_and_uncs("p")[0]**2*1e6

In [ ]:
get_params_and_uncs("d")[0]

In [ ]:
transits = [f"Transit {i}" for i in range(1, 6)]

def get_params_and_uncs(param):
    ps, p_us, p_ds = [], [], []
    for transit in transits:
        df_transit = df_results[transit]
        p, p_u, p_d = df_transit.loc[param][["Value", "SigmaUp", "SigmaDown"]]
        ps.append(p)
        p_us.append(p_u)
        p_ds.append(p_d)
    return np.array(ps), np.array(p_us), np.array(p_ds)

#### Corner Plot

In [6]:
import astropy.constants as c

In [ ]:
#set_theme('paper')
# Load
data_dir = "/home/mango/ACCESS_notebook/Projects/HATP26b/data_detrending"
#data_dir = "data/data_detrending/WASP43"
#data_dir = 'data/data_detrending/HATP23_c'
binsize = 'custom'
fpath_truths = f'{data_dir}/truth.json'
with open(fpath_truths) as f:
    params_dict = json.load(f)

def write_latex(row):
    v, vu, vd = row
    return f'{v:.4f}^{{+{vu:.4f}}}_{{-{vd:.4f}}}'

data_dict = {
    "Transit 1":
    f"{data_dir}/hp26_190313_c/white-light/BMA_posteriors.pkl",
}

"""
data_dict = {
    'Transit 1':
    f'{data_dir}/w43_150224/white-light/BMA_posteriors.pkl',
    
    'Transit 2':
    f'{data_dir}/w43_150309/white-light/BMA_posteriors.pkl',
    
    'Transit 3':
    f'{data_dir}/w43_170410/white-light/BMA_posteriors.pkl',
    
    'Transit 4':
    f'{data_dir}/w43_180603/white-light/BMA_posteriors.pkl',
}
"""

"""
data_dict = {
    'Transit 1':
    f'{data_dir}/hp23b_160621_{binsize}/white-light/BMA_posteriors.pkl',
    
    'Transit 2':
    f'{data_dir}/hp23b_170609_{binsize}/white-light/BMA_posteriors.pkl',
    
    'Transit 3':
    f'{data_dir}/hp23b_180603_{binsize}/white-light/BMA_posteriors.pkl',
    
    'Transit 4':
    f'{data_dir}/hp23b_180620_{binsize}/white-light/BMA_posteriors.pkl',
    
    'Transit 5':
    f'{data_dir}/hp23b_180821_{binsize}/white-light/BMA_posteriors.pkl',
    
    #'$aR_\mathrm{s} + (\mathrm{b}, \mathrm{p})$':
    #f'{data_dir}/out_ab/HATP23b/hp23b_180603_{binsize}/white-light/BMA_posteriors.pkl',
    #
    #'$aR_\mathrm{s} + (r_1, r_2)$':
    #f'{data_dir}/out_ar/HATP23b/hp23b_180603_{binsize}/white-light/BMA_posteriors.pkl',
    #
    #'$\\rho_\mathrm{s} + (\mathrm{b}, \mathrm{p})$':
    #f'{data_dir}/out_rb/HATP23b/hp23b_180603_{binsize}/white-light/BMA_posteriors.pkl',
    #
    #'$\\rho_\mathrm{s} + (r_1, r_2)$':
    #f'{data_dir}/out_rr/HATP23b/hp23b_180603_{binsize}/white-light/BMA_posteriors.pkl',
    #
    #'py2':
    #f'{data_dir}/out_py2/HATP23b/hp23b_180603_{binsize}/white-light/BMA_posteriors.pkl',
}
"""

# Plot
fig = None # Initialize figure
for t_i, (transit_name, fpath) in enumerate(data_dict.items()):
    # Load
    data = utils.pkl_load(fpath)
    if 'rho' not in data.keys():
        G = c.G.cgs.value
        aR = data['aR']
        P = data['P']
        data['rho'] = 3.0*np.pi * aR**3 / (G*(P*86400.0)**2)
    samples = pd.DataFrame(data)[params_dict.keys()]
    if 'WASP43' not in data_dir:
        samples['t0'] -= 2450000
    
    mins = np.min(np.array([samples.min(), samples.min()]), axis=0)
    maxs = np.min(np.array([samples.max(), samples.max()]), axis=0)
    ranges = list(zip(mins, maxs))

    # Plot
    fig, axes = utils.plot_corner(
        samples,
        fpath_truths,
        c=f'C{t_i}',
        fig=fig,
        ranges=ranges,
    )
    
    # Custom titles
    ps = [0.16, 0.5, 0.84]
    ps_strs = [f'{p*100:.0f}%' for p in ps]
    df_stats = samples.describe(percentiles=ps).loc[ps_strs]
    df_latex = pd.DataFrame(columns=df_stats.columns)
    df_latex.loc['p'] = df_stats.loc['50%']
    df_latex.loc['p_u'] = df_stats.loc['84%'] - df_stats.loc['50%']
    df_latex.loc['p_d'] = df_stats.loc['50%'] - df_stats.loc['16%']
    
    titles = df_latex.apply(write_latex, axis=0).to_list()
    
    ndim = samples.shape[1]
    axes = np.array(fig.axes).reshape((ndim, ndim))
    for i, (param_key, param_data) in enumerate(params_dict.items()):
        ax = axes[i, i] # select 1d hist
        ax.annotate(
            f'${titles[i]}$',
            xy=(0.0, 1.1 + t_i/4.0),
            xycoords='axes fraction',
            ha="left",
            color=f'C{t_i}',
            fontsize=14,
        )
    
# Label custom titles
for i, (param_key, param_data) in enumerate(params_dict.items()):
    ax = axes[i, i] # select 1d hist
    ax.annotate(
        f'{param_data["symbol"]}',
        xy=(0.5, 1.1 + (t_i+1)/4.0),
        xycoords='axes fraction',
        ha='center',
        #color='w',
    )  
    p_mean, p_u, p_d = param_data["truth"] # Unpack mean +/-
    ax.axvspan(p_mean - p_u, p_mean + p_u, alpha=0.75, color='grey',
               lw=0, zorder=0)
    ax.axvline(p_mean, color='w')

# True values
with open(fpath_truths) as f:
    params_dict = json.load(f)
truths = [v['truth'][0] for v in params_dict.values()]
ndim = len(truths)
for yi in range(ndim):
    for xi in range(yi):
        ax = axes[yi, xi]
        ax.plot(truths[xi], truths[yi], "P", ms=10, mec="grey", mfc="w")

# Custom legend
handles, labels = [], []
for transit_name in list(data_dict.keys()):
    dummy, = plt.plot([], [], 'D', ms=12, mfc='none', mew=2, label=transit_name)
    handles.append(dummy)
    labels.append(transit_name)
fig.legend(handles, labels, loc=1, fontsize=18, ncol=len(data_dict)+1)  

fig.set_size_inches(14, 14)
# Save
#utils.savefig(f"projects/HATP23b/paper/figures/detrended_wlcs_corners/corner_wlcs.pdf")
#title = data_dir.split('/')[-1]
#utils.savepng("/home/mango/Projects/HATP26b/journal/figures/wlc_corner")

In [ ]:
def get_quantiles(dist,alpha = 0.68, method = 'median'):
    """ 
    get_quantiles function
    DESCRIPTION
        This function returns, in the default case, the parameter median and the error% 
        credibility around it. This assumes you give a non-ordered 
        distribution of parameters.
    OUTPUTS
        Median of the parameter,upper credibility bound, lower credibility bound
    """
    ordered_dist = dist[np.argsort(dist)]
    param = 0.0 
    # Define the number of samples from posterior
    nsamples = len(dist)
    nsamples_at_each_side = int(nsamples*(alpha/2.)+1)
    if(method == 'median'):
        med_idx = 0 
        if(nsamples%2 == 0.0): # Number of points is even
            med_idx_up = int(nsamples/2.)+1
            med_idx_down = med_idx_up-1
            param = (ordered_dist[med_idx_up]+ordered_dist[med_idx_down])/2.
            return param,ordered_dist[med_idx_up+nsamples_at_each_side],\
                   ordered_dist[med_idx_down-nsamples_at_each_side]
        else:
            med_idx = int(nsamples/2.)
            param = ordered_dist[med_idx]
            return param,ordered_dist[med_idx+nsamples_at_each_side],\
                   ordered_dist[med_idx-nsamples_at_each_side]

In [ ]:
mu = 1.2128867
sigma = 0.0000002
x1 = 1.21288287
x2 = mu + 2*sigma

z1 = ( x1 - mu ) / sigma
z2 = ( x2 - mu ) / sigma

x = np.arange(z1, z2, 0.001) # range of x in spec
x_all = np.arange(-10, 10, 0.001) # entire range of x, both in and out of spec
# mean = 0, stddev = 1, since Z-transform was calculated
y = sp.stats.norm.pdf(x, 0, 1)
y2 = sp.stats.norm.pdf(x_all, 0, 1)

fig, ax = plt.subplots(figsize=(9,6))
ax.axvline(z1)
ax.axvline(z2)
ax.plot(x_all,y2)
ax.fill_between(x, y, 0, alpha=0.3, color='b')
#ax.fill_between(x_all, y2, 0, alpha=0.1)
#ax.set_xlim([-4,4])
ax.set_xlabel('# of Standard Deviations Outside the Mean')
ax.set_yticklabels([])
ax.set_title('Normal Gaussian Curve')

In [ ]:
1.212880 - 3*0.000002

In [ ]:
ps = [0.16, 0.5, 0.84]
ps_strs = [f'{p*100:.0f}%' for p in ps]
df_stats = samples_new.describe(percentiles=ps).loc[ps_strs]
df_latex = pd.DataFrame(columns=df_stats.columns)
df_latex.loc['p'] = df_stats.loc['50%']
df_latex.loc['p_u'] = df_stats.loc['84%'] - df_stats.loc['50%']
df_latex.loc['p_d'] = df_stats.loc['50%'] - df_stats.loc['16%']
df_latex

In [ ]:
get_quantiles(samples_new['inc'])

In [ ]:
data_dirs = sorted(glob.glob('data/data_detrending/HATP23_*'))

for data_dir in data_dirs:
    print(data_dir)
    fpaths = glob.glob(f'{data_dir}/hp*/white-light/PCA_*/posteriors_trend_george.pkl')

    for fpath in sorted(fpaths):
        lnZ = utils.pkl_load(fpath)['lnZ']
        print(lnZ)
    print()

### Binned LC

In [ ]:
##############
# Read in data
##############
data = {"Transit 1":"190313"}
# data = {
#     'Transit 1':'160621',
#     'Transit 2':'170609',
#     'Transit 3':'180603',
#     'Transit 4':'180620',
#     'Transit 5':'180821',
# }
#binsize = "custom"
for title, date in data.items():
    #GPT_dir = f"data/data_detrending/HATP23_c/hp23b_{date}_{binsize}"
    GPT_dir = "./data/data_detrending/HATP26_25nm/hp26_190313_25nm"
    def wbin_num(fpath):
        # Extracts <num> from fpath = .../wbin<num>/...
        tokens = fpath.split('/')
        for token in tokens:
            if "wbin" in token: # Do the extraction
                bin_str = token.split("wbin")[-1]
                bin_num = int(bin_str)
                return(bin_num)
    wbin_paths = sorted(glob.glob(f'{GPT_dir}/wavelength/wbin*'), key=wbin_num)
    PCA_list = []
    for wbin_path in wbin_paths:
        PCA_paths = glob.glob(f'{wbin_path}/PCA*')
        PCAs = [path.split('/')[-1] for path in PCA_paths]
        PCA_list.append(PCAs)

    common_PCAs = set(PCA_list[0])
    for s in PCA_list[1:]:
        common_PCAs.intersection_update(s)

    PCA_max = max(common_PCAs, key=lambda s: int(s.split('_')[-1]))
    PCA_num = int(PCA_max.split('_')[-1])
    print(f'max common PCA = {PCA_num}')

    # getting t0 from WLC data
    def get_result(fpath, key="t0", unc=True):
        data = np.genfromtxt(fpath, encoding=None, dtype=None)
        for line in data:
            if key in line: 
                if unc: return line
                else: return line[1]

        print(f"'{key}' not found. Check results.dat file.")        
    fpath = f"{GPT_dir}/white-light/results.dat"
    t0 = float(get_result(fpath, key="t0", unc=False))
    P = float(get_result(fpath, key="P", unc=False))

    # Get wavelength bins
    fpath = f"{GPT_dir}/transpec.csv"
    wbins = np.loadtxt(fpath, skiprows=1, usecols=[0, 1], delimiter=',')

    # Glob doesn't automatically sort, but instead follows your local filesystem's 
    # rules, which can be very system dependent. 
    # To avoid potential cross-platform issues, I just sort based on an explicit
    # rule that is passed to `sorted`. In this case, the rule is: 
    # sort based on the <num> part in wbin<num> of each file path.
    dirpath = f"{GPT_dir}/wavelength"
    detrended_files = f"{dirpath}/wbin*/PCA_{PCA_num}/detrended_lc.dat"
    fpaths = sorted(glob.glob(detrended_files), key=wbin_num)

    # Store final data in <# of wavelength bins> x <length of timeseries> arrays 
    detfluxes, models, resids = [], [], []
    for fpath in fpaths:
        time, detflux, detfluxerr, model = np.loadtxt(fpath, unpack=True)
        detfluxes.append(detflux)
        models.append(model)
        resids.append(detflux - model + 1)
    detfluxes = np.array(detfluxes).T
    models = np.array(models).T
    resids = np.array(resids).T
    phase = utils.get_phases(time, P, t0)
    time_rel = phase*24 #(time - t0)*24 # Convert to hours

    ###################################
    # Plot detrended flux and residuals
    ###################################
    # Plot configs
    import matplotlib.patheffects as PathEffects
    N = detfluxes.shape[1] # number of wavelength bins
    colors = np.array(sns.color_palette("Spectral_r", N))

    offset = 0.014 # 0.01 # spacing betweem binned lcs
    # Optional bins to highlight
    species = {
        'Na I-D':5892.9,
        #'Hα':6564.6,
        'K I':7682.0,
        'Na I-8200':8189.0
    }
    scatter_plot_kwargs = {
        "marker":".", 
        "lw":0, 
        "mew":0, # Make non-zero to show marker outlines
        #"mec":'k', # Won't show if `mew`=0
    }
    annotate_kwargs = {
        "fontsize":8, 
        "horizontalalignment":'right',
        "path_effects":[PathEffects.withStroke(linewidth=1, foreground="k")],
    }
    annotate_rms_kwargs = {
        "fontsize":8, 
        "horizontalalignment":'left',
        "path_effects":[PathEffects.withStroke(linewidth=1, foreground="k")],
    }

    fig, axes = plt.subplots(1, 2, figsize=(8, 11), sharex=True, sharey=True)
    ax_left, ax_right = axes.flatten()

    # detrended flux
    ax_left.set_title('Detrended flux')
    p_det = utils.plot_binned(ax_left, time_rel, detfluxes, wbins, offset, colors, 
                              plot_kwargs=scatter_plot_kwargs, models=models)

    # residual flux
    ax_right.set_title('Residuals')
    baselines = np.ones_like(resids)
    p_res = utils.plot_binned(
        ax_right,
        time_rel,
        resids,
        wbins,
        offset,
        colors, 
        plot_kwargs=scatter_plot_kwargs,
        models=baselines,
        annotate=True,
        annotate_kwargs=annotate_kwargs,
        annotate_rms_kwargs=annotate_rms_kwargs,
        species=species,
    ) 

    ax_left.set_ylabel("Normalized flux + offset")
    fig.text(0.47, 0, 'Phase (hours)', ha='left')

    ax_left.set_xlim(-1, 1)

    fig.set_size_inches(8, 8)
    fig.tight_layout()

    #plt.savefig("/Users/mango/Desktop/wlc_access.png", dpi=250, bbox_inches="tight")

    title = title.lower().replace(' ', '_') + '_detr_blcs'
    #utils.savefig(f"projects/HATP23b/paper/figures/detrended_blcs/{title}.pdf")
    utils.savepng("/home/mango/Projects/HATP26b/journal/blcs")

max common PCA = 2


### Transmission spectrum

In [18]:
set_theme("dark")
#data_dir = 'data/data_detrending/HATP23_c'
#data_dir = "data/data_detrending/WASP43"
data_dir = "/home/mango/ACCESS_notebook/Projects/HATP26b/data_detrending"
binsize = 'custom'
if data_dir.split('/')[-1].split('_')[-1] == 's':
    binsize = 'species'
"""
data_dict = {
    #'Transit 1':{
    #'path_tspec':f'{data_dir}/hp23b_160621_{binsize}/transpec.csv',
    #'path_wlc':f'{data_dir}/hp23b_160621_{binsize}/white-light/results.dat',
    #},
    
    'Transit 1':f'{data_dir}/hp23b_160621_{binsize}',
    
    'Transit 2':f'{data_dir}/hp23b_170609_{binsize}',
    
    'Transit 3':f'{data_dir}/hp23b_180603_{binsize}',
    
    'Transit 4':f'{data_dir}/hp23b_180620_{binsize}',
    
    'Transit 5':f'{data_dir}/hp23b_180821_{binsize}',
    
    #'$aR_\mathrm{s} + (\mathrm{b}, \mathrm{p})$':f'{data_dir}/out_ab/HATP23b/hp23b_180603_{binsize}',
    
    #'$\\rho_\mathrm{s} + (\mathrm{b}, \mathrm{p})$':f'{data_dir}/out_rb/HATP23b/hp23b_180603_{binsize}',
    
    #'py2':f'{data_dir}/out_py2/HATP23b/hp23b_180603_{binsize}',
}
"""

# data_dict = {
#     #"Transit 1":f"{data_dir}/w43_150224",
#     "Transit 2":f"{data_dir}/w43_150309",
#     "Transit 3":f"{data_dir}/w43_170410",
#     "Transit 4":f"{data_dir}/w43_180603",
# }

data_dict = {
    "Transit 1":f"{data_dir}/hp26_190313_c5"
}

def write_latex(row):
    v, vu, vd = row
    return f'{v:.3f}^{{+{vu:.3f}}}_{{-{vd:.3f}}}'
def write_latex_wav(row):
    wav_d, wav_u = row
    return f'{wav_d:.1f} - {wav_u:.1f}'

# Use first entry for wavelength
transit_0, dirpath_0 = next(iter(data_dict.items()))
fpath = f'{dirpath_0}/transpec.csv'
df_wavs = pd.read_csv(fpath)[['Wav_d', 'Wav_u']]
wav = (df_wavs['Wav_u'] + df_wavs['Wav_d']) / 2.0

depth_wlc_stats = []
tspec_stats = []
for transit, dirpath in data_dict.items():
    print(transit)
    # WLCs
    fpath = f'{dirpath}/white-light/results.dat'
    #p_stats = pd.read_table(fpath, sep='\s+', escapechar='#').query('` Variable` == "p"')
    p_stats = pd.read_table(fpath, sep='\s+', escapechar='#').query('Variable == "p"')
    p, p_u, p_d = p_stats[['Value', 'SigmaUp', 'SigmaDown']].values[0].T
    wlc_depth = p**2 * 1e6
    wlc_depth_u = p_u**2 * 1e6
    wlc_depth_d = p_d**2 * 1e6
    depth_wlc_stats.append([wlc_depth, wlc_depth_u, wlc_depth_d])
    
    # Tspec
    fpath = f'{dirpath}/transpec.csv'
    df_tspec = pd.read_csv(fpath)[
        ['Depth (ppm)', 'Depthup (ppm)', 'DepthDown (ppm)']
    ]
        
    if transit == "Transit 1" and binsize=="custom":
        tspec, tspec_u, tspec_d = df_tspec.values.T
    else:
        tspec, tspec_u, tspec_d = df_tspec.values[1:-1, :].T
        
    tspec_stats.append([tspec, tspec_u, tspec_d])
    
# Compute offset
depth_wlc_stats = np.array(depth_wlc_stats)
depth_wlc, depth_wlc_u, depth_wlc_d = depth_wlc_stats.T
mean_wlc_depth, mean_wlc_depth_unc = utils.weighted_mean_uneven_errors(
    depth_wlc, depth_wlc_u, depth_wlc_d
)

wlc_offsets = depth_wlc - mean_wlc_depth
print(f"offsets: {wlc_offsets}")
print(f"offsets (% mean wlc depth): {wlc_offsets*100/mean_wlc_depth}")
tspec_stats = np.array(tspec_stats) # transits x (depth, u, d) x wavelength
tspec_stats[:, 0, :] -= wlc_offsets[np.newaxis].T

tspec_depths = tspec_stats[:, 0, :]
tspec_us = tspec_stats[:, 1, :]
tspec_ds = tspec_stats[:, 2, :]

# Plot
fig, ax = plt.subplots()
ax.axhline(mean_wlc_depth, color="darkgrey", zorder=0, ls='--')

# Species
species = {
    'Na I-D':5892.9, 
    #'Hα':6564.6, 
    'K I_avg':7682.0, 
    'Na I-8200_avg':8189.0
}
[ax.axvline(wav, ls='--', lw=0.5, color='grey', zorder=0) for name, wav in species.items()]

tspec_tables = {} # Each entry will hold Table(Transit, Depth, Up , Down)
# For combining into latex later
#colors = ['C0', 'C2', 'C4']
#for transit, tspec, tspec_d, tspec_u, c in zip(
for transit, tspec, tspec_d, tspec_u, in zip(
    data_dict.keys(),
    tspec_depths,
    tspec_ds,
    tspec_us,
    #colors,
):
    ax.errorbar(
        wav,
        tspec,
        #xerr=[wav - df_wavs['Wav_d'], df_wavs['Wav_u'] - wav],
        yerr=[tspec_d, tspec_u],
        fmt='o',
        alpha=1.0,
        mew=0,
        label=transit,
        barsabove=False,
        #color=c,
    )
    
    data_i = {}
    data_i['Depth (ppm)'] = tspec
    data_i['Depthup (ppm)'] = tspec_u
    data_i['DepthDown (ppm)'] = tspec_d
    df = pd.DataFrame(data_i)
    tspec_tables[transit] = df

tspec_combined = []
tspec_combined_unc = []
tspec_combined_max = []
tspec_combined_unc_max = []
for i in range(len(tspec_stats[0, 0, :])):
    tspec_comb, tspec_comb_unc = utils.weighted_mean_uneven_errors(
        tspec_stats[:, 0, i],
        tspec_stats[:, 1, i],
        tspec_stats[:, 2, i],
    )
    tspec_combined.append(tspec_comb)
    tspec_combined_unc.append(tspec_comb_unc)
        
    # single errorbar way
    uncs_max = np.max([tspec_stats[:, 1, i], tspec_stats[:, 2, i]], axis=0)
    weights = 1 / uncs_max**2
    tspec_comb_max = np.average(tspec_stats[:, 0, i], weights=weights)
    tspec_comb_max_unc = utils.weighted_err(uncs_max)
    tspec_combined_max.append(tspec_comb_max)
    tspec_combined_unc_max.append(tspec_comb_max_unc)

# Combined
# tspec_combined = np.array(tspec_combined)
# tspec_combined_unc = np.array(tspec_combined_unc)
# p = ax.errorbar(
#     wav,
#     tspec_combined,
#     yerr=tspec_combined_unc,
#     c='w',
#     mec='k',
#     fmt='o',
#     zorder=10,
#     label="combined",
#     ecolor='k',
#     lw=4,
# )
#fpath = "projects/HATP23b/data/tspec/tspec_combined.dat"
#fpath = "/home/mango/Desktop/yea.dat"
#np.savetxt(fpath, np.c_[wav/1e4, tspec_combined*1e-6, tspec_combined_unc*1e-6])

# Write to table
tspec_table = pd.DataFrame()
tspec_table['Wavelength (Å)'] = df_wavs.apply(write_latex_wav, axis=1)
# Transmission spectra
for transit, df_tspec in tspec_tables.items():
    tspec_table[transit] = df_tspec.apply(write_latex, axis=1)
data = np.array([tspec_combined, tspec_combined_unc]).T
df_combined = pd.DataFrame(data, columns=["Combined", "Unc"])
def write_latex(row):
    v, v_unc = row
    return f'{v:.3f} \pm {v_unc:.3f}'
tspec_table['Combined'] = df_combined.apply(write_latex, axis=1)
ax.legend(ncol=6, loc=1, fontsize=12, frameon=True)
#ax.set_xlim(5106.55, 9362.45)
#ax.set_ylim(9_000, 17_000)

# Inset plots
if binsize=="species":
    margin=10
    utils.plot_inset(
        ax,
        species_slc=slice(0,5),
        box_lims=[0.3, 0.15, 0.2, 0.2],
        lims=(5780.40-margin, 6005.40+margin),
    )
    utils.plot_inset(
        ax,
        species_slc=slice(5,10),
        box_lims=[0.38, 0.65, 0.2, 0.2],
        lims=(7657-margin, 7707+margin),
    )
    utils.plot_inset(
        ax,
        species_slc=slice(10,15),
        box_lims=[0.78, 0.15, 0.2, 0.2],
        lims=(8089-margin, 8289+margin),
    )

title = "tspec"
if binsize == "species":
    title = "tspec_species"
    # Plot inset for species fig
    # [x0, y0, width, height] relative to lower left corner
    # Shortcut to zoom in on last instrument
    #axins.set_xlim(p.get_xlim())
    #axins.set_ylim(p.get_ylim())
    #plot_model(axins, model, model_kwargs=model_kwargs, fill_kwargs=fill_kwargs)
    
#ax.set_title("ACCESS Magellan/IMACS Transit Spectra of HAT-P-23")
ax.set_xlabel('Wavelength (Å)')
ax.set_ylabel(r'Transit Depth (ppm)')
fig.set_size_inches(FIG_WIDE)
fig.tight_layout()
#utils.savefig(f'projects/HATP23b/paper/figures/tspec/{title}.pdf')
#utils.savepng('/home/mango/Projects/HATP26b/journal/figures/tspec')

print("mean WLC depth:", mean_wlc_depth, mean_wlc_depth_unc)
Rs = 0.96 * u.solRad
Rp = np.sqrt(mean_wlc_depth*1e-6 * Rs**2)
Mp = 1.35 * u.jupiterMass
gp = c.G * Mp  /Rp**2
print("Rs (Rsun):", Rs.to("Rsun"))
print("Rp (Rj):", Rp.to("Rjupiter"))
print("gp (m/s^2):", gp.to("m/s^2"))

Transit 1
offsets: [0.89061188]
offsets (% mean wlc depth): [0.01685872]
mean WLC depth: 5282.796590975366 11.006123230623828


AttributeError: 'numpy.ndarray' object has no attribute 'G'

In [ ]:
Transit 1
offsets: [-0.28737234]
offsets (% mean wlc depth): [-0.00514713]
mean WLC depth: 5583.157185935618 12.071465288531083
Rs (Rsun): 0.96 solRad
Rp (Rj): 0.69803261828029 jupiterRad
gp (m/s^2): 68.67487561900433 m / s2

In [ ]:
fpath = f'{dirpath}/white-light/results.dat'
p_stats = pd.read_table(fpath, sep='\s+', escapechar='#').query('` Variable` == "p"')
p_stats

In [ ]:
fpath = f'{dirpath}/transpec.csv'
df_tspec = pd.read_csv(fpath)
df_tspec

In [ ]:
for transit, table in tspec_tables.items():
    mi = min(table[['Depthup (ppm)', 'DepthDown (ppm)']].min(axis=0)),
    ma = max(table[['Depthup (ppm)', 'DepthDown (ppm)']].max(axis=0)),
    print(transit, mi, ma)

In [ ]:
table

In [ ]:
tspec_table.to_clipboard(index=False)

In [ ]:
data_max = np.array([tspec_combined_max, tspec_combined_unc_max]).T
df_combined_max = pd.DataFrame(data_max, columns=["Combined Max", "Unc Max"])
def write_latex(row):
    v, v_unc = row
    return f'{v:.5f} \pm {v_unc:.5f}'
#df_combined.apply(write_latex, axis=1)
df_combined_max

In [ ]:
weights

In [ ]:
data = {
    '#Wlow':df_wavs['Wav_d'],
    'Wup':df_wavs['Wav_u'],
    'Depth':df_combined['Combined'],
    'ErrUp':df_combined['Unc'],
    'ErrLow':df_combined['Unc'],
    'Instrument':'Magellan/IMACS',
    'Offset?':'NO',
}
df_retrieval = pd.DataFrame(data)
df_retrieval.to_csv(
    'projects/HATP23b/data/retrieval/tspec_hp23_c.csv',
    index=False,
)
df_retrieval

## XO

### WLC

#### LC

In [ ]:
dirpath = "xo/HATP23/ut180603/wlc"
lc = np.load(f"{dirpath}/lc.npy")
map_soln = utils.pkl_load(f"{dirpath}/map_soln.pkl")

fig, ax = plt.subplots(figsize=FIG_WIDE)
ax.plot(map_soln["light_curve"] + 1.0020465216919012, label="MAP")
ax.plot(lc[:, 1], '.', label=dirpath)
ax.legend()

In [ ]:
np.load("xo/HATP23/ut180603/wlc_bak/lc.npy").shape

In [ ]:
dirpath = "xo/HATP23/ut180603/wlc_bak"
fig, axes = utils.plot_exoplanet_WLC(dirpath, figsize=FIG_LARGE)
#fig.suptitle(dirpath)
#plt.savefig("/Users/mango/Desktop/HATP23_ut180603.png", dpi=250, bbox_inches="tight")
#plt.savefig("/Users/mango/Desktop/w43_ut180603_xo_wlc.svg", bbox_inches="tight")

#### Corner plot

In [ ]:
dirpath = "xo/HATP23/ut1806"
fig, axes, df_samples = utils.plot_corner(
    dirpath=dirpath,
    title=""" "truth" values from Sada & Ramón-Fox (2016) """,
    #title=""" "truth" values from Weaver et al. (2020) """,
)

#xlims = np.array([[ax.get_xlim() for ax in ax_row] for ax_row in axes])
#ylims = np.array([[ax.get_ylim() for ax in ax_row] for ax_row in axes])

"""
for ax_row, xlim_row, ylim_row in zip(axes, xlims, ylims):
    for ax, xlim, ylim in zip(ax_row, xlim_row, ylim_row):
        ax.set_xlim(xlim)
        ax.set_ylim(ylim)
"""
        
#plt.savefig(f"{dirpath}/corner.png", dpi=250, bbox_inches="tight")
#plt.savefig(f"/Users/mango/Desktop/corner_hp23_ut180603.png", dpi=250, bbox_inches="tight")
plt.savefig(f"/Users/mango/Desktop/hp23_ut180603_xo_wlc_corner.png", dpi=250, bbox_inches="tight")

#### Trace plot

In [ ]:
fpath = "xo/WASP43/ut180603/wl_bak/trace.pkl"
trace = utils.pkl_load(fpath)

In [ ]:
trace.keys()

In [ ]:
r_traces = [trace["trace"][i]['r'] for i in range(1000)]

In [ ]:
plt.hist(r_traces)

### Binned

In [ ]:
fpaths = sorted(glob.glob("xo/WASP43/ut180603/binned_free_rho_star/wbin_*/detrended.csv"))
columns = {"flux", "lc_model"}
DT_list = [dt.fread(fpath, columns=columns) for fpath in fpaths]
phase = dt.fread(fpaths[0], columns={"phase"}).to_numpy()
fluxes = dt.cbind(*[DT["flux"] for DT in DT_list]).to_numpy()
models = dt.cbind(*[DT["lc_model"] for DT in DT_list]).to_numpy()
resids = fluxes - models + 1.
#df_list = [DT.to_pandas() for DT in DT_list]
#fluxes = dt.cbind(*DT_list).to_numpy()
#models = 

In [ ]:
fpath_tepspec = "data/data_reductions/WASP43/ut180603_a9_24_noflat_LBR/LCs_w43_kreidberg.pkl"


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(11, 11), sharex=True, sharey=True)
ax_left, ax_right = axes
N_bins = fluxes.shape[1]
wbins = utils.pkl_load(fpath_tepspec)["wbins"]
wbins = np.array(wbins)
idxs_used = phase

# Plot binned light curves
p_left = utils.plot_binned(
        ax_left,
        idxs_used,
        fluxes,
        utc=False,
        bins=wbins, 
        offset=0.01,
        colors=np.array(sns.color_palette("Spectral_r", N_bins)),
        plot_kwargs={"marker":'.', "mew":0, "lw":0},
        models=models,
    )
p_left.set_title("detrended flux - exoplanet")

# TODO: Try detrending with `exoplanet`
# Plot residuals
p_right = utils.plot_binned(
        ax_right,
        idxs_used,
        resids,
        utc=False,
        bins=wbins, 
        offset=0.01,
        colors=np.array(sns.color_palette("Spectral_r", N_bins)),
        plot_kwargs={"marker":'.', "mew":0, "lw":0},
        annotate=True,
        annotate_kwargs={"fontsize":10, "ha":"center"},
    )
p_right.set_title("Residuals")

ax_left.set_ylabel("Normalized flux + offset")
fig.text(0.5, 0, 'Time (JD)', ha='left')

fig.tight_layout()

plt.savefig("/Users/mango/Desktop/binned_xo.png", dpi=250, bbox_inches="tight")

### Transmission spectra

In [ ]:
fpaths = sorted(glob.glob("xo/WASP43/ut180603/binned_free_rho_star/wbin_*/summary.csv"))
columns = slice(0, 3) #{"C0", "mean", "sd"}
DT_list = [dt.fread(fpath, columns=columns)[dt.f["C0"] == "r", :][:, ["mean","sd"]] for fpath in fpaths]

In [ ]:
tspec = dt.rbind(*DT_list).to_numpy()

fig, ax = plt.subplots(figsize=(11, 6))
ax.errorbar(range(len(df["Rp/Rs"])), df["Rp/Rs"], yerr=df["Rp/RsErrUp"], fmt='o',
            label="GPTS (square_exp)", color='b', alpha=0.5)
ax.errorbar(range(len(tspec[:, 0])), tspec[:, 0], yerr=tspec[:, 1], fmt='o',
            label="exoplanet-quick (matern)", color='C1', alpha=0.5)
ax.legend(loc=2)

ax.set_xlabel("index")
ax.set_ylabel(r"$R_p/R_s$")
ax.grid(axis="x")
fig.tight_layout()
plt.savefig("/Users/mango/Desktop/tspec.png", dpi=250, bbox_inches="tight")

In [ ]:
sns.palplot(sns.color_palette())

In [ ]:
columns = {"flux", "lc_model"}
DT_list = [dt.fread(fpath, columns=columns) for fpath in fpaths]
phase = dt.fread(fpaths[0], columns={"phase"}).to_numpy()
fluxes = dt.cbind(*[DT["flux"] for DT in DT_list]).to_numpy()
models = dt.cbind(*[DT["lc_model"] for DT in DT_list]).to_numpy()
resids = fluxes - models + 1.
#df_list = [DT.to_pandas() for DT in DT_list]
#fluxes = dt.cbind(*DT_list).to_numpy()
#models = 

# Forward model

In [ ]:
np.sqrt(1e-6*12_945 * 1.152**2) * 9.731

In [ ]:
# Common directory
dirpath = "projects/HATP23b/data"

# Models
models = {
    "generic_local_001_056_0001_000.dat":"normal scattering, clear",
    "generic_local_001_056_0001_001.dat":"normal scattering, cloudy",
    "generic_local_001_056_1100_000.dat":"enhanced scattering, clear",
    "generic_local_001_056_1100_001.dat":"enhanced scattering, cloudy",
    "hatp23b_solar_noTiOVO.dat":"cold (no TiO/VO)",
    "generic_detr.dat":"detr",
}

# Plot models
fig, ax = plt.subplots(figsize=FIG_WIDE)
for model, l in models.items():
    fpath = f"{dirpath}/forward_model/{model}"
    wav_model, flux_model = np.loadtxt(fpath, unpack=True)
    ax.plot(wav_model*1e4, flux_model*1e6, label=l)

# Load data
data = "tspec/tspec_combined.dat"
fpath = f"{dirpath}/{data}"
wav_data, flux_data, flux_data_err = np.loadtxt(fpath, unpack=True)

# Plot data
ax.plot(wav_data, flux_data, 'o')
ax.errorbar(
    wav_data*1e4,
    flux_data*1e6,
    yerr=flux_data_err*1e6,
    c='w',
    mec='k',
    fmt='o',
    zorder=10,
    label="this study",
    ecolor='k',
    lw=2,
)

ax.legend(fontsize=12, ncol=3)
ax.set_xlim(5106.55, 9362.45)
ax.set_ylim(9_000, 17_000)
ax.set_xlabel("Wavelength (Å)")
ax.set_ylabel("Transit depth (ppm)")
fig.tight_layout()

# Retrieval

## exoretrievals

### Plot

In [ ]:
set_theme('paper')

fig, ax = plt.subplots()

########
# Models
########
dirpath = "data/retrievals/HATP23_wider"
model_key_name = "Na+K+TiO (haze)"
models = {
    "K (clear)":f"{dirpath}/HATP23_E1_NoHet_FitP0_NoClouds_NoHaze_fitR0_K",
    f"{model_key_name}":f"{dirpath}/HATP23_E1_NoHet_FitP0_NoClouds_Haze_fitR0_Na_K_TiO",
}

instruments = {
    'Magellan_IMACS': {
        'c': 'w', 'mec':'k', 'fmt': 'o', 
        'ecolor':'k', 
        #'lw':4,
        'label': 'Magellan/IMACS',
        'zorder':10,
    },
}
colors = ["grey", "C5"]
for i, (model_name, model_path) in enumerate(models.items()):
    # Plot model
    model = ascii.read(f'{model_path}/retr_model.txt')
    model_kwargs = {'label':model_name, "alpha":1}
    fill_kwargs = {'alpha':0.125, "color":colors[i]}
    _, p = utils.plot_model(
        ax, model, model_kwargs=model_kwargs, fill_kwargs=fill_kwargs,
    )
    
    # Plot sampled points
    for instrument in instruments.keys():
        instr_sampled = ascii.read(f'{model_path}/retr_model_sampled_{instrument}.txt')
        sampled_kwargs = {'marker':'s', 'color':p[0].get_color(), 'lw':0, "alpha":1}
        utils.plot_instrument(
            ax, instr_sampled=instr_sampled, sampled_kwargs=sampled_kwargs,
        )

#############
# Instruments
#############
# Dict of instr_kwargs
instruments = {
    'Magellan_IMACS': {
        'c': 'w', 'mec':'k', 'fmt': 'o', 
        'ecolor':'k', 
        #'lw':4,
        'label': 'Magellan/IMACS',
        'zorder':10,
    },
}
model_path = models['K (clear)'] # Instrument sampling identical for all models
for instrument, instr_kwargs in instruments.items():
    instr = ascii.read(f'{model_path}/retr_{instrument}.txt')
    instrument_name = instrument.replace('_', '/')
    utils.plot_instrument(
        ax, instr, sampled=False, instr_kwargs=instr_kwargs,
    )
    
####################
# Annotate Delta lnZ
####################
fpath = f'{models[model_key_name]}/retrieval.pkl'
fpath_flat = f'{models["K (clear)"]}/retrieval.pkl'
DlnZ, DlnZ_unc, lnZ, lnZ_unc = utils.get_Delta_lnZ(fpath, fpath_flat)
s = f"$\Delta \ln(Z) = {DlnZ:.2f} \pm {DlnZ_unc:.2f}$"
ax.annotate(s, (0.02, 0.05), xycoords='axes fraction')

"""
# Plot inset
axins = ax.inset_axes([0.5, 0.5, 0.5, 0.5])
# Shortcut to zoom in on last instrument
p = plot_instrument(axins, instr, instr_sampled, instr_kwargs=configs, sampled_kwargs=sampled_kwargs)
axins.set_xlim(p.get_xlim())
axins.set_ylim(p.get_ylim())
plot_model(axins, model, model_kwargs=model_kwargs, fill_kwargs=fill_kwargs)
ax.indicate_inset_zoom(axins, alpha=1.0, edgecolor='w')
"""

ax.set_xlim(0.5, 0.95)
#ax.set_ylim(25000, 27000)
ax.set_ylim(11_000, 16_000)

ax.set_xlabel(r'Wavelength $(\mu\mathrm{m})$')
ax.set_ylabel('Transit depth (ppm)')
ax.legend(loc=1, ncol=3)

#########
# Species
#########
species = {
    'Na I-D':5892.9, 
    #'Hα':6564.6, 
    'K I_avg':7682.0, 
    'Na I-8200_avg':8189.0
}
[ax.axvline(wav/10000, lw=0.5, ls='--', color='grey', zorder=0) for name, wav in species.items()]

#ax.annotate(f"Python 2", (0.05, 0.9), xycoords='axes fraction')

#plt.savefig('/Users/mango/Desktop/exoretrievals_py2.png', dpi=250, bbox_inches='tight')

fig.set_size_inches(FIG_WIDE)
#fig.tight_layout()
#utils.savefig('projects/HATP23b/paper/figures/retrievals/retrieval.pdf')
plt.savefig(f'/Users/mango/Desktop/tspec_haze_mid.pdf', bbox_inches='tight')
#plt.savefig(f'../retrieval/kreidberg/{species}/tspec/retr_{basename}.pdf', bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots()

p = ax.fill_between([1,2,3,4,5], [5,5,5,5,5], [1,1,1,1,1])

### Yee

In [ ]:
p.get_facecolor()[0]

In [ ]:
dirpath.split('E1_')[-1].split('FitP0')[0] + dirpath.split('/')[-1].split('R0_')[-1]

In [ ]:
kwargs = {}
def test(x, y, **kwargs): 
    return x + y
    
plt.plot([1,2,3], **{})

### Retrieval evidences

In [ ]:
dirpath = 'data/retrievals/HATP23'
species = ["Na", "K", "TiO", "Na_K", "Na_TiO", "K_TiO", "Na_K_TiO"]

data_dict = {
    sp:{
        'clear':f'{dirpath}/HATP23_E1_NoHet_FitP0_NoClouds_NoHaze_fitR0_{sp}',
        'haze':f'{dirpath}/HATP23_E1_NoHet_FitP0_NoClouds_Haze_fitR0_{sp}',
        'spot':f'{dirpath}/HATP23_E1_Het_FitP0_NoClouds_NoHaze_fitR0_{sp}',
        'spot+haze':f'{dirpath}/HATP23_E1_Het_FitP0_NoClouds_Haze_fitR0_{sp}',
    }
    for sp in species
}

#dirpath_flat = f'{dirpath}/HATP23_E1_NoHet_FlatLine'
dirpath_flat = f'{dirpath}/HATP23_E1_NoHet_FitP0_NoClouds_NoHaze_fitR0_K'
fpath_flat = f'{dirpath_flat}/retrieval.pkl'
data = {}
for species, model_info in data_dict.items():
    data[species] = {}
    for model, dirpath in model_info.items():
        fpath = f'{dirpath}/retrieval.pkl'
        Delta_lnZ, Delta_lnZ_unc, lnZ, lnZ_unc = utils.get_Delta_lnZ(
            fpath, fpath_flat
        )
        data[species][model] = utils.write_latex2(
            Delta_lnZ, Delta_lnZ_unc)
#         data[species][model] = (lnZ, lnZ_unc)

df_retr = pd.DataFrame(data)
df_retr#.to_clipboard()

In [ ]:
df_retr.min(axis=1)

### Corner Plot

In [ ]:
set_theme("paper")
fpath = 'data/retrievals/HATP23/HATP23_E1_Het_FitP0_NoClouds_NoHaze_fitR0_Na_K_TiO/retrieval.pkl'
post = utils.pkl_load(fpath)

df = pd.DataFrame(post['samples']) 

params = {
    'logP0':r'$\log P_0$',
    'T':r'$T_\mathrm{p}$',
#     'logH2O':r'$\log \mathrm{H}_2\mathrm{O}$',
    'logNa':r'$\log \mathrm{Na}$',
    'logK':r'$\log \mathrm{K}$',
    'logTiO':r'$\log \mathrm{TiO}$',
    'f':r'$f$',
}

if "_Haze" in fpath:
    params['loga'] = r'$\log a$'
    params['gamma'] = r'$\gamma_\mathrm{haze}$'

if "_Het" in fpath:
    params['Tocc'] = r'$T_\mathrm{star}$'
    params['Thet'] = r'$T_\mathrm{het}$'
    params['Fhet'] = r'$f_\mathrm{het}$'

df_params = df[params.keys()]

corner_kwargs = {
    'show_titles':True,
}
hist_kwargs = {'histtype':'stepfilled', 'lw':2, 'density':True,}
fig, axes = utils.plot_corner(
    df_params,
    params=params,
    c=f'C5',
    corner_kwargs=corner_kwargs,
    hist_kwargs=hist_kwargs,
)

fig.set_size_inches(18, 18)

utils.savefig('projects/HATP23b/paper/figures/retrievals/retrieval_corner.pdf')
#plt.savefig(f'../retrieval/kreidberg/{species}/corner/corner_{basename}.pdf', bbox_inches='tight')
#plt.savefig(f'/Users/mango/Desktop/corner_{source}_all.pdf', bbox_inches='tight')
#plt.savefig(f"/Users/mango/Desktop/corner_test.pdf", bbox_inches="tight")

## CHIMERA

### Transmisson Spectrum File

In [ ]:
d, h = fits.getdata(fpath, header=True)

Puts transmission into format that can be read by CHIMERA

In [ ]:
fpath = "../WASP43/retrieval/kreidberg/GP_kr_noffsH.dat"
df_exoretrievals = pd.read_csv(fpath, sep='\s+')
df_exoretrievals.rename({"#Wlow":"Wlow"}, axis=1, inplace=True)

# create CHIMERA datatable
df_CHIMERA = pd.DataFrame()

# add mid-wavelength column
wav_low_AA, wav_up_AA = df_exoretrievals[["Wlow", "Wup"]].T.values
df_CHIMERA["wl [um]"] = (wav_low_AA + wav_up_AA)/2 * 1e-4
df_CHIMERA["wl [um]"] = df_CHIMERA["wl [um]"].apply(lambda x: f"{x:.18e}")

# add (Rp/Rstar)^2
depth = df_exoretrievals["Depth"]
df_CHIMERA["(Rp/Rstar)^2"] = depth*1e-6
df_CHIMERA["(Rp/Rstar)^2"] = df_CHIMERA["(Rp/Rstar)^2"].apply(lambda x: f"{x:.18e}")


# add (Rp/Rstar)^2 err
errup_ppm, errlow_ppm = df_exoretrievals[["ErrUp", "ErrLow"]].T.values
err_ppm = (errlow_ppm + errlow_ppm)/2
df_CHIMERA["(Rp/Rstar)^2 err"] = err_ppm * 1e-6
df_CHIMERA["(Rp/Rstar)^2 err"] = df_CHIMERA["(Rp/Rstar)^2 err"].apply(lambda x: f"{x:.18e}")

# write to txt file
fpath = "/Users/mango/Desktop/w43b_trans.txt"

with open(fpath, 'w') as file:
    header = "#Weaver et al. 2019\n#wl [um]\t\t\t(Rp/Rstar)^2\t\t(Rp/Rstar)^2 err\n"
    file.write(header)
    df_CHIMERA.to_csv(file, sep=" ", header=False, index=False)

### View Output

In [ ]:
#fpath = "../data/WASP43/test/spectral_samples_trans_pmn_wfc3_cc.pic"
fpath = "../data/WASP43/test/pmn_transmission_wfc3_cc.pic"
data = utils.pkl_load(fpath)

In [ ]:
data.shape

In [ ]:
plt.plot(data[:, 0])